In [ ]:
# # ============================================================
# # 📚 全流程模块（DeepSeek + OpenRouter/Claude 3.7 双后端）
# #   1. 拆章节 split_novels()
# #   2. 批量 LLM 分析 process()   → 章级 *_processed.txt
# #   3. 扩展：provider 可选 "deepseek" / "openrouter"
# #      - 两端均支持 response_format={"type":"json_object"}
# # ============================================================

# # ── 安装必需包 ────────────────────────────────────────────────
# !pip -q install --upgrade openai tqdm chardet pandas matplotlib jieba networkx requests

# # ── 通用库 ───────────────────────────────────────────────────
# import os, re, json, logging, unicodedata, chardet, requests
# from pathlib import Path
# from typing import List, Dict, Tuple
# from collections import defaultdict
# from concurrent.futures import ThreadPoolExecutor
# from tqdm.auto import tqdm

# # ── DeepSeek / OpenRouter 配置 ───────────────────────────────
# class Provider(str):
#     DEEPSEEK   = "deepseek"
#     OPENROUTER = "openrouter"

# DEEPSEEK_API_KEY  = os.getenv("DEEPSEEK_API_KEY")   or "your-default-api-key"
# DEEPSEEK_URL      = "https://api.deepseek.com"
# DEEPSEEK_MODEL    = "deepseek-chat"

# OPENROUTER_API_KEY= os.getenv("OPENROUTER_API_KEY") or "your-default-api-key"
# OPENROUTER_URL    = "https://openrouter.ai/api/v1/chat/completions"
# OPENROUTER_MODEL  = "anthropic/claude-3.7-sonnet"

# import openai                                # 只给 DeepSeek 用
# deep_client = openai.OpenAI(api_key=DEEPSEEK_API_KEY, base_url=DEEPSEEK_URL)

# logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")

# # ============================================================
# # 1️⃣ 拆章节 split_novels()
# # ============================================================
# _CHAPTER_PAT = re.compile(
#     r"""
#     ^\s*(
#         第[\d零一二三四五六七八九十百千万]+\s*[章节卷回]\s* |
#         [零一二三四五六七八九十百千万]{1,4}[\.．、\s]+ |
#         (?:Chapter|CHAPTER)\s+\d+               |
#         \d{1,3}[\.．、]\s*                      |
#         \d{1,3}\s+
#     )\s*(.*?)$
#     """, re.MULTILINE | re.IGNORECASE | re.VERBOSE
# )

# def _safe_name(s: str) -> str:
#     s = unicodedata.normalize("NFKC", s)
#     s = re.sub(r"[\r\n\t]+", " ", s)
#     s = re.sub(r'[\\/:*?"<>|]', "_", s)
#     s = re.sub(r"\s+", "", s)
#     return s[:80] or "未知"

# def _auto_decode(path: Path) -> str:
#     raw = path.read_bytes()
#     enc = chardet.detect(raw)["encoding"] or "utf-8"
#     return raw.decode(enc, errors="ignore")

# def split_novels(input_dir: str, output_base: str | None = None) -> Dict[str, List[Path]]:
#     in_p, out_p = Path(input_dir), Path(output_base or f"{input_dir.rstrip('/')}_chapters")
#     out_p.mkdir(parents=True, exist_ok=True)

#     novels: Dict[str, List[Path]] = {}
#     for txt in in_p.glob("*.txt"):
#         book_dir = out_p / _safe_name(txt.stem); book_dir.mkdir(exist_ok=True)
#         data = _auto_decode(txt)
#         ms = list(_CHAPTER_PAT.finditer(data))
#         blocks = [(ms[i].group().strip(),
#                    data[ms[i].end():ms[i+1].start()] if i+1<len(ms) else data[ms[i].end():])
#                   for i in range(len(ms))] if ms else \
#                  [(f"未知章节{i+1}", data[s:s+5000]) for i,s in enumerate(range(0,len(data),5000))]
#         paths=[]
#         for i,(title,body) in enumerate(blocks,1):
#             p=book_dir/f"{i:03d}_{_safe_name(title)}.txt"
#             p.write_text(body.strip(),encoding="utf-8"); paths.append(p)
#         novels[book_dir.name]=paths
#         logging.info(f"《{txt.stem}》→ {len(paths)} 章")
#     return novels
# #============================================================ 
# # 2️⃣ 统一 LLM 调用 _call_llm()
# # ============================================================
# def _call_llm(prompt:str, content:str,
#               *, provider:Provider=Provider.DEEPSEEK,
#               json_mode:bool=False) -> str:
#     if provider==Provider.DEEPSEEK:
#         rsp = deep_client.chat.completions.create(
#             model=DEEPSEEK_MODEL,
#             messages=[{"role":"system","content":prompt},
#                       {"role":"user","content":content}],
#             temperature=0.3,
#             max_tokens=4096,
#             response_format={"type":"json_object"} if json_mode else None
#         )
#         return rsp.choices[0].message.content.strip()

#     if provider==Provider.OPENROUTER:
#         body = {
#             "model": OPENROUTER_MODEL,
#             "messages":[
#                 {"role":"system","content":prompt},
#                 {"role":"user","content":content}
#             ]
#         }
#         if json_mode:
#             body["response_format"]={"type":"json_object"}
#         r = requests.post(OPENROUTER_URL,
#                           headers={
#                               "Authorization":f"Bearer {OPENROUTER_API_KEY}",
#                               "Content-Type":"application/json"
#                           },
#                           data=json.dumps(body))
#         if r.status_code!=200:
#             raise RuntimeError(f"OpenRouter {r.status_code}: {r.text[:200]}")
#         return r.json()["choices"][0]["message"]["content"].strip()

#     raise ValueError("Unknown provider")

# # ============================================================
# # 3️⃣ process_chapters() / process()
# # ============================================================
# def process_chapters(chapter_files: List[Path],
#                      prompt: str,
#                      *,
#                      provider:Provider = Provider.DEEPSEEK,
#                      json_mode: bool = True,
#                      workers:int = 4,
#                      suffix="_processed.txt"):
#     def _run(fp:Path):
#         res=_call_llm(prompt,fp.read_text(encoding="utf-8"),
#                       provider=provider,json_mode=json_mode)
#         out=fp.with_name(fp.stem+suffix); out.write_text(res,encoding="utf-8")
#         return out
#     with ThreadPoolExecutor(max_workers=workers) as ex:
#         list(ex.map(_run, chapter_files))

# def _gather(root:Path)->List[Path]:
#     return sorted(root.rglob("*.txt"))

# def process(chapter_root:str,*,
#             prompt:str,
#             provider:Provider=Provider.DEEPSEEK,
#             out_dir:str="/content/json_results",
#             json_mode:bool=True,
#             chapters:Tuple[int,int]|None=None):
#     s,e = chapters or (1,float("inf"))
#     root, out_base = Path(chapter_root), Path(out_dir); out_base.mkdir(parents=True,exist_ok=True)
#     books = [d for d in root.iterdir() if d.is_dir()] or [root]

#     for book in tqdm(books, desc="📚 书库"):
#         files=[p for p in _gather(book) if s<=int(p.stem.split("_")[0])<=e]
#         for group_idx in range(0,len(files),10):
#             grp=files[group_idx:group_idx+10]
#             if not grp: continue
#             start=int(grp[0].stem[:3]); end=start+len(grp)-1
#             sub=out_base/book.name/f"{start:03d}-{end:03d}"; sub.mkdir(parents=True,exist_ok=True)
#             todo=[p for p in grp if not (sub/f"{p.stem}_processed.txt").exists()]
#             if todo:
#                 process_chapters(todo,prompt,
#                                  provider=provider,json_mode=json_mode)
#                 for tp in todo:
#                     (sub/f"{tp.stem}_processed.txt").write_text(
#                         (tp.parent/f"{tp.stem}_processed.txt").read_text(encoding="utf-8"),
#                         encoding="utf-8")
#                     (tp.parent/f"{tp.stem}_processed.txt").unlink()
#     print(f"✅ 输出目录：{out_base}")


# def run_analysis(
#     chapter_root: str,
#     *,
#     prompt: str,
#     provider: Provider = Provider.DEEPSEEK,
#     out_dir: str = "/content/json_results",
#     mode: str | tuple[int, int] = "full",   # ← 关键参数
#     json_mode: bool = True,
# ):
#     """
#     mode 用法
#     --------
#     • "full"          → 分析整本
#     • 100             → 只分析 1~100 章
#     • (51, 150)       → 分析 51~150 章
#     """
#     if mode == "full":
#         chapters = None
#     elif isinstance(mode, int):
#         chapters = (1, mode)
#     elif isinstance(mode, tuple) and len(mode) == 2:
#         chapters = mode
#     else:
#         raise ValueError("mode 必须是 'full'、整数 N，或 (start, end) 元组")

#     process(
#         chapter_root=chapter_root,
#         prompt=prompt,
#         provider=provider,
#         out_dir=out_dir,
#         json_mode=json_mode,
#         chapters=chapters
#     )

In [ ]:
# # ============================================================
# # 📊 小说章节 JSON 统计（库级批量版，区间压缩 + 保存）
# # ============================================================
# import json, re
# from pathlib import Path
# from collections import defaultdict
# from typing import Dict, List, Tuple

# import pandas as pd
# import matplotlib.pyplot as plt
# from IPython.display import display

# # ---------- 工具 ----------
# _DIGIT_RE = re.compile(r"(\d{3})")
# def _extract_no(p: Path) -> int:
#     m = _DIGIT_RE.search(p.stem) or re.match(r"(\d{3})-", p.parent.name)
#     return int(m.group(1)) if m else -1

# def _compress(nums: List[int]) -> List[str]:
#     if not nums: return []
#     nums = sorted(nums)
#     res, s, prev = [], nums[0], nums[0]
#     for n in nums[1:]:
#         if n == prev + 1: prev = n; continue
#         res.append(f"{s}-{prev}" if s != prev else f"{s}")
#         s = prev = n
#     res.append(f"{s}-{prev}" if s != prev else f"{s}")
#     return res

# # ---------- 单本 ----------
# # ---------- 修正版 stat_book ----------
# # ---------- 工具：出现映射 ----------
# def _map_occ(df: pd.DataFrame, col: str) -> Dict[str, List[int]]:
#     mp = defaultdict(list)
#     for _, row in df.iterrows():
#         for item in row[col]:
#             mp[item].append(row["chapter"])
#     return mp

# def _build(mp: Dict[str,List[int]], *, min_chaps=1, top_n=20):
#     data = [
#         {"name": k, "count": len(v), "chapters": _compress(v)}
#         for k, v in mp.items() if len(v) >= min_chaps
#     ]
#     return sorted(data, key=lambda x: x["count"], reverse=True)[:top_n]

# # ---------- 单本统计 ----------
# def stat_book(book_dir: str | Path,
#               *,
#               min_chapters: int = 3,
#               top_n: int = 30,
#               show_plot: bool = False) -> dict:
#     """对单本小说目录生成统计 dict"""
#     book_dir = Path(book_dir)
#     rows, auto_idx = [], 1

#     for p in sorted(book_dir.rglob("*_processed.txt")):
#         chap_no = _extract_no(p)
#         if chap_no == -1:
#             chap_no, auto_idx = auto_idx, auto_idx + 1   # 顺序补号
#         try:
#             data = json.loads(p.read_text(encoding="utf-8"))
#         except json.JSONDecodeError:
#             print(f"⚠️ 跳过无效 JSON: {p}"); continue

#         rows.append({
#             "chapter":   chap_no,
#             "characters": data.get("出现人物", []),
#             "scenes":     data.get("出现场景", []),
#             "props":      data.get("出现道具", []),
#             "setup":      data.get("伏笔_设下", []),
#             "recycle":    data.get("伏笔_回收", []),
#         })

#     if not rows:
#         raise ValueError(f"{book_dir} 无有效 *_processed.txt")

#     df = pd.DataFrame(rows).sort_values("chapter").reset_index(drop=True)  # ← 这行之前缺失

#     char_map = _map_occ(df, "characters")
#     major    = {k: v for k, v in char_map.items() if len(v) >= min_chapters}

#     summary = {
#         "主要角色": _build(major, min_chaps=min_chapters, top_n=top_n),
#         "场景":     _build(_map_occ(df, "scenes"),  top_n=top_n),
#         "道具":     _build(_map_occ(df, "props"),   top_n=top_n),
#         "伏笔_设下": _build(_map_occ(df, "setup"),  top_n=top_n),
#         "伏笔_回收": _build(_map_occ(df, "recycle"),top_n=top_n),
#     }

#     if show_plot and major:
#         plt.figure(figsize=(10, 4))
#         for c, chs in major.items():
#             plt.plot(df["chapter"], [1 if n in chs else 0 for n in df["chapter"]], label=c)
#         plt.xlabel("章节号"); plt.ylabel("出现(1)"); plt.title(book_dir.name); plt.legend()
#         plt.show()

#     return summary

# # ---------- 整库批量 ----------
# def stat_library(root_dir: str | Path,
#                  out_root: str | Path = "/content/novel_stats",
#                  *, min_chapters: int = 3,
#                  top_n: int = 10,
#                  show_plot: bool = False):
#     root_dir = Path(root_dir); out_root = Path(out_root)
#     out_root.mkdir(parents=True, exist_ok=True)

#     for book in root_dir.iterdir():
#         if not book.is_dir(): continue
#         try:
#             res = stat_book(book,
#                             min_chapters=min_chapters,
#                             top_n=top_n,
#                             show_plot=show_plot)
#         except ValueError as e:
#             print(e); continue

#         out_file = out_root / book.name / f"{book.name}_stats.json"
#         out_file.parent.mkdir(parents=True, exist_ok=True)
#         out_file.write_text(json.dumps(res, ensure_ascii=False, indent=2), encoding="utf-8")
#         print(f"✅ 写入 {out_file.relative_to(out_root)}")

In [ ]:
# # 1. 拆章节
# # split_novels("/content/novels", "/content/novels_chapters")

# # 2. 定义章节分析 Prompt,有两种，json的和plain的，400字的和800字的，200字的。
# # prompt_chapter_json = r"""
# # 你是一位专业文学编辑。请阅读我接下来提供的【章节全文】，按照下列要求生成 **400-500 字** 的详细章节分析，并 **仅以 JSON 格式** 返回结果（严禁输出任何解释或 Markdown）。

# # JSON 顶层字段与要求：
# # {
# #   "章节定位导语":         "<100-200 字，说明章节在整书中的位置和作用>",
# #   "情节摘要导语":         "<200-300 字，概括该章节的主要内容,请尽量把剧情讲述完整>",
# #   "情感与节奏导语":       "<200-300 字，分析章节的情感和节奏变化>",
# #   "关键场景分析":         "<200-400 字，深入分析章节中的重要场景>",
# #   "人物角色变化":         "<200-300 字，分析章节中人物的表现和变化>",
# #   "情感张力变化":         "<100-200 字，分析章节情感的起伏>",
# #   "节奏与结构观察":       "<100-200 字，分析章节节奏与整书主题和情节的关联>",
# #   "出现人物":             ["角色A", "角色B", ...],
# #   "出现道具":             ["道具1", "道具2", ...],
# #   "出现场景":             ["场景1", "场景2"],
# #   "伏笔_设下":            ["伏笔1 描述", "伏笔2 描述"],
# #   "伏笔_回收":            ["伏笔A 回收方式", ...]
# # }

# # 严格要求：
# # 1. 文本字段须为完整中文段落；字数必须落在区间内。
# # 2. 上述 12 个键一个都不能少，也不能多。
# # 3. “出现人物 / 道具 / 场景 / 伏笔” 用 JSON 数组。
# # 4. 可引用原文≤10%，需加引号注明段落号。
# # 【章节全文】：
# # """

# # prompt_chapter_json_100000_json = r"""
# # 你是一位专业文学编辑。请阅读我接下来提供的【章节全文】，按照下列要求生成 **400-500 字** 的详细章节分析，并 **仅以 JSON 格式** 返回结果（严禁输出任何解释或 Markdown）。

# # JSON 顶层字段与要求：
# # {
# #   "章节定位导语":         "<40 字，说明章节在整书中的位置和作用>",
# #   "情节摘要导语":         "<40 字，概括该章节的主要内容,请尽量把剧情讲述完整>",
# #   "情感与节奏导语":       "<40 字，分析章节的情感和节奏变化>",
# #   "关键场景分析":         "<40 字，深入分析章节中的重要场景>",
# #   "人物角色变化":         "<40 字，分析章节中人物的表现和变化>",
# #   "情感张力变化":         "<40 字，分析章节情感的起伏>",
# #   "节奏与结构观察":       "<40 字，分析章节节奏与整书主题和情节的关联>",
# #   "出现人物":             ["角色A", "角色B", ...],
# #   "出现道具":             ["道具1", "道具2", ...],
# #   "出现场景":             ["场景1", "场景2"],
# #   "伏笔_设下":            ["伏笔1 描述", "伏笔2 描述"],
# #   "伏笔_回收":            ["伏笔A 回收方式", ...]
# # }

# # 严格要求：
# # 1. 文本字段须为完整中文段落；字数必须落在区间内。
# # 2. 上述 12 个键一个都不能少，也不能多。
# # 3. “出现人物 / 道具 / 场景 / 伏笔” 用 JSON 数组。
# # 4. 可引用原文≤10%，需加引号注明段落号。
# # 【章节全文】：
# # """


# prompt_chapter_json_100000_json = r"""
# 你是一位专业文学编辑。请阅读我接下来提供的【章节全文】，按照下列要求生成总计 **400-500 字** 的详细章节分析，并 **仅以 JSON 格式** 返回结果（严禁输出任何解释或 Markdown）。

# JSON 顶层字段与要求：
# {
#   "情节摘要导语":         "<400 字，概括该章节的主要内容,请尽量把剧情讲述完整>",
#   "出现人物":             ["角色A", "角色B", ...],
#   "出现道具":             ["道具1", "道具2", ...],
#   "出现场景":             ["场景1", "场景2"],
#   "伏笔_设下":            ["伏笔1 描述", "伏笔2 描述"],
#   "伏笔_回收":            ["伏笔A 回收方式", ...]
# }

# 严格要求：
# 1. 文本字段须为完整中文段落；字数必须落在区间内。
# 2. 上述 6 个键一个都不能少，也不能多。
# 3. “出现人物 / 道具 / 场景 / 伏笔” 用 JSON 数组。
# 4. 可引用原文≤10%，需加引号注明段落号。
# 【章节全文】：
# """

# prompt_chapter_json_50000_json = r"""
# 你是一位专业文学编辑。请阅读我接下来提供的【章节全文】，按照下列要求生成总计 **200-250 字** 的详细章节分析，并 **仅以 JSON 格式** 返回结果（严禁输出任何解释或 Markdown）。

# JSON 顶层字段与要求：
# {
#   "情节摘要导语":         "<200 字，概括该章节的主要内容,请尽量把剧情讲述完整>",
#   "出现人物":             ["角色A", "角色B", ...],
#   "出现道具":             ["道具1", "道具2", ...],
#   "出现场景":             ["场景1", "场景2"],
#   "伏笔_设下":            ["伏笔1 描述", "伏笔2 描述"],
#   "伏笔_回收":            ["伏笔A 回收方式", ...]
# }

# 严格要求：
# 1. 文本字段须为完整中文段落；字数必须落在区间内。
# 2. 上述 6 个键一个都不能少，也不能多。
# 3. “出现人物 / 道具 / 场景 / 伏笔” 用 JSON 数组。
# 4. 可引用原文≤10%，需加引号注明段落号。
# 【章节全文】：
# """


# prompt_chapter_json_200000_json = r"""
# 你是一位专业文学编辑。请阅读我接下来提供的【章节全文】，按照下列要求生成总计 **700-800 字** 的详细章节分析，并 **仅以 JSON 格式** 返回结果（严禁输出任何解释或 Markdown）。

# JSON 顶层字段与要求：
# {
#   "情节摘要导语":         "<700 字，概括该章节的主要内容,请尽量把剧情讲述完整>",
#   "出现人物":             ["角色A", "角色B", ...],
#   "出现道具":             ["道具1", "道具2", ...],
#   "出现场景":             ["场景1", "场景2"],
#   "伏笔_设下":            ["伏笔1 描述", "伏笔2 描述"],
#   "伏笔_回收":            ["伏笔A 回收方式", ...]
# }

# 严格要求：
# 1. 文本字段须为完整中文段落；字数必须落在区间内。
# 2. 上述 6 个键一个都不能少，也不能多。
# 3. “出现人物 / 道具 / 场景 / 伏笔” 用 JSON 数组。
# 4. 可引用原文≤10%，需加引号注明段落号。
# 【章节全文】：
# """


# # 3. 批量分析（只处理 1–100 章）
# # 只分析 1–100 章
# # process(
# #     chapter_root="/content/novels_chapters",
# #     prompt=prompt_chapter_json,
# #     out_dir="/content/json_results",
# #     json_mode=True,
# #     chapters=(1, 10)
# # )

# # # 1) 分析前 150 章（DeepSeek）
# run_analysis(
#     chapter_root="/content/novels_chapters",
#     prompt=prompt_chapter_json_50000_json,
#     provider=Provider.DEEPSEEK,
#     out_dir="/content/json_results/50000_json",
#     mode=(1, 150)      # 指定区间
# )

# run_analysis(
#     chapter_root="/content/novels_chapters",
#     prompt=prompt_chapter_json_100000_json,
#     provider=Provider.DEEPSEEK,
#     out_dir="/content/json_results/100000_json",
#     mode=(1, 150)      # 指定区间
# )

# run_analysis(
#     chapter_root="/content/novels_chapters",
#     prompt=prompt_chapter_json_200000_json,
#     provider=Provider.DEEPSEEK,
#     out_dir="/content/json_results/200000_json",
#     mode=(1, 150)      # 指定区间
# )

In [ ]:
# # =============================
# # ✅ Gemini 分析小说章节，结构化输出 JSON（强制调用函数）
# # =============================
# !pip install -q --upgrade google-generativeai chardet tqdm 
# !pip install -U google-generativeai


# import os, json, re, time, random, string, chardet, logging
# from pathlib import Path
# from typing import List
# from tqdm.auto import tqdm
# import google.generativeai as genai

# # —— API 初始化 ——
# os.environ["GEMINI_API_KEY"]     = "your-api-key"     # ★必填用 Gemini

# GEMINI_MODEL = "gemini-2.0-flash"

# genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
# gemini_model = genai.GenerativeModel(GEMINI_MODEL)

# logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")

# # —— 通用函数 ——
# def _auto_decode(path: Path) -> str:
#     raw = path.read_bytes()
#     enc = chardet.detect(raw)["encoding"] or "utf-8"
#     return raw.decode(enc, errors="ignore").strip()

# def _rand_tag(k=6): return ''.join(random.choices(string.ascii_uppercase, k=k))

# # —— JSON 输出字段要求（6 项） ——
# json_schema = {
#     "type": "object",
#     "properties": {
#         "情节摘要导语": {"type": "string"},
#         "出现人物": {"type": "array", "items": {"type": "string"}},
#         "出现道具": {"type": "array", "items": {"type": "string"}},
#         "出现场景": {"type": "array", "items": {"type": "string"}},
#         "伏笔_设下": {"type": "array", "items": {"type": "string"}},
#         "伏笔_回收": {"type": "array", "items": {"type": "string"}}
#     },
#     "required": ["情节摘要导语", "出现人物", "出现道具", "出现场景", "伏笔_设下", "伏笔_回收"]
# }

# # —— Prompt 模板（可替换为你指定的 50k / 100k / 200k） ——
# prompt_chapter_json = r"""
# 你是一位专业文学编辑。请阅读我接下来提供的【章节全文】，按照下列要求生成总计 **400-500 字** 的详细章节分析，并 **仅以 JSON 格式** 返回结果（严禁输出任何解释或 Markdown）。

# JSON 顶层字段与要求：
# {
#   "情节摘要导语": "<400 字，概括该章节的主要内容,请尽量把剧情讲述完整>",
#   "出现人物": ["角色A", "角色B", ...],
#   "出现道具": ["道具1", "道具2", ...],
#   "出现场景": ["场景1", "场景2"],
#   "伏笔_设下": ["伏笔1 描述", "伏笔2 描述"],
#   "伏笔_回收": ["伏笔A 回收方式", ...]
# }

# 严格要求：
# 1. 文本字段须为完整中文段落；字数必须落在区间内。
# 2. 上述 6 个键一个都不能少，也不能多。
# 3. “出现人物 / 道具 / 场景 / 伏笔” 用 JSON 数组。
# 4. 可引用原文≤10%，需加引号注明段落号。
# 【章节全文】：
# """

# # —— 分析单章（使用工具函数 + 强制调用） ——
# def analyze_chapter(path: Path, prompt: str, retries: int = 3) -> str:
#     text = _auto_decode(path)
#     base_prompt = f"{prompt.strip()}\n\n#TAG:{_rand_tag()}"

#     for attempt in range(retries):
#         try:
#             rsp = gemini_model.generate_content(
#                 [base_prompt, text],
#                 generation_config={"temperature": 0.3, "max_output_tokens": 8096},
#                 tools=[{
#                     "function_declarations": [
#                         {
#                             "name": "analyze_chapter",
#                             "description": "提取小说章节结构化信息（摘要、人物、道具、场景、伏笔）",
#                             "parameters": json_schema
#                         }
#                     ]
#                 }],
#                 tool_choice={"function_call": {"name": "analyze_chapter"}}  # ✅ 强制调用
#             )
#             fc = rsp.candidates[0].content.parts[0].function_call
#             return json.dumps(fc.args, ensure_ascii=False, indent=2)
#         except Exception as e:
#             logging.warning(f"Gemini 错误重试: {e}")
#             time.sleep(2 ** attempt)

#     raise RuntimeError(f"❌ 分析失败：{path.name}")

# # —— 批量处理目录下所有章节 ——
# def run_analysis(
#     chapter_root: str,
#     prompt: str,
#     out_dir: str,
#     mode: tuple[int, int] = (1, 150)
# ):
#     root = Path(chapter_root)
#     out_base = Path(out_dir)
#     out_base.mkdir(parents=True, exist_ok=True)

#     books = [d for d in root.iterdir() if d.is_dir()]
#     s, e = mode

#     for book in tqdm(books, desc="📚 小说书目"):
#         chapters = sorted([p for p in book.glob("*.txt")
#                            if s <= int(p.stem[:3]) <= e])
#         out_book_dir = out_base / book.name
#         out_book_dir.mkdir(parents=True, exist_ok=True)

#         for chap in tqdm(chapters, desc=f"📖 {book.name}"):
#             out_path = out_book_dir / f"{chap.stem}_processed.txt"
#             if out_path.exists(): continue
#             try:
#                 result = analyze_chapter(chap, prompt)
#                 out_path.write_text(result, encoding="utf-8")
#             except Exception as e:
#                 logging.warning(f"❌ {chap.name} 分析失败: {e}")

#     print(f"\n✅ 所有章节结构化完成，结果保存在：{out_base}")

In [ ]:
# # 1) 分析 200–250 字版本（可用于中短章）
# run_analysis(
#     chapter_root="/content/novels_chapters",
#     prompt=prompt_chapter_json.replace("400-500", "200-250").replace("<400", "<200"),
#     out_dir="/content/json_results/50000_json_gemini"
# )

# # 2) 分析 400–500 字版本（标准分析）
# run_analysis(
#     chapter_root="/content/novels_chapters",
#     prompt=prompt_chapter_json,
#     out_dir="/content/json_results/100000_json_gemini"
# )

# # 3) 分析 700–800 字版本（超长章节）
# run_analysis(
#     chapter_root="/content/novels_chapters",
#     prompt=prompt_chapter_json.replace("400-500", "700-800").replace("<400", "<700"),
#     out_dir="/content/json_results/200000_json_gemini"
# )

In [ ]:
# # =============================
# # ✅ Gemini 分析小说章节，结构化输出 JSON（强制调用函数 + 可变摘要长度 + JSON修复 + 硬编码Key）
# # =============================
# # !pip install -q --upgrade google-generativeai chardet tqdm
# # !pip install -U google-generativeai # 确保安装最新版本

# import os
# import json
# import re
# import time
# import random
# import string
# import chardet
# import logging
# import copy  # 导入 copy
# from pathlib import Path
# from typing import List, Dict, Any
# from tqdm.auto import tqdm
# import google.generativeai as genai
# from google.generativeai.types import HarmCategory, HarmBlockThreshold

# # —— API 初始化 (⚠️ 警告：直接写入 API 密钥极不安全！) ——
# # 强烈建议您使用环境变量或 Secrets Manager 等更安全的方式管理 API 密钥。
# # 直接将密钥写入代码会带来严重的安全风险，尤其是在共享或版本控制代码时。

# # ⚠️ 请将下面的 "YOUR_API_KEY_HERE" 替换为您真实的 Gemini API 密钥！
# api_key = "your-default-api-key"

# # 基本检查，确保用户替换了占位符
# if not api_key or api_key == "YOUR_API_KEY_HERE":
#     raise ValueError("❌ 错误：请务必将代码中的 'YOUR_API_KEY_HERE' 替换为您的真实 Gemini API 密钥。")

# # 使用直接写入的密钥配置 SDK
# try:
#     genai.configure(api_key=api_key)
#     logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s") # Setup logging after configure
#     logging.info("Gemini API 已使用直接写入的密钥进行配置。")
# except Exception as e:
#     # 如果配置失败，提前记录错误并退出
#     # logging 可能尚未完全配置，尝试打印错误
#     print(f"CRITICAL: 使用提供的 API 密钥配置 Gemini SDK 时出错: {e}")
#     raise ValueError(f"API 密钥配置失败: {e}")

# # 使用推荐的最新 Flash 模型
# GEMINI_MODEL = "gemini-2.0-flash"

# # —— 通用函数 ——
# def _auto_decode(path: Path) -> str:
#     """自动检测文件编码并读取内容"""
#     try:
#         raw = path.read_bytes()
#         enc = chardet.detect(raw)["encoding"] or "utf-8"
#         # 增加对常见中文编码的尝试
#         if enc.lower() not in ['utf-8', 'gbk', 'gb2312', 'big5']:
#              try: return raw.decode('utf-8', errors='ignore').strip()
#              except UnicodeDecodeError:
#                  try: return raw.decode('gbk', errors='ignore').strip()
#                  except UnicodeDecodeError: return raw.decode(enc, errors='ignore').strip() # 最后尝试 chardet 的结果
#         return raw.decode(enc, errors="ignore").strip()
#     except Exception as e:
#         logging.error(f"读取文件 {path.name} 时出错: {e}")
#         return "" # 返回空字符串，让后续处理知道失败

# def _rand_tag(k=6):
#     """生成随机标签（可选）"""
#     return ''.join(random.choices(string.ascii_uppercase, k=k))

# # —— 基础 JSON 输出字段要求 ——
# base_json_schema = {
#     "type": "object",
#     "properties": {
#         "情节摘要导语": {"type": "string", "description": ""}, # 描述将在调用时动态设置
#         "出现人物": {"type": "array", "items": {"type": "string"}, "description": "本章出现的所有人物名称列表"},
#         "出现道具": {"type": "array", "items": {"type": "string"}, "description": "本章出现的关键道具列表"},
#         "出现场景": {"type": "array", "items": {"type": "string"}, "description": "本章发生故事的主要场景列表"},
#         "伏笔_设下": {"type": "array", "items": {"type": "string"}, "description": "本章新埋下的伏笔描述"},
#         "伏笔_回收": {"type": "array", "items": {"type": "string"}, "description": "本章回收或呼应的过往伏笔描述"}
#     },
#     "required": ["情节摘要导语", "出现人物", "出现道具", "出现场景", "伏笔_设下", "伏笔_回收"]
# }

# # —— Prompt 模板 (强调遵循参数描述) ——
# prompt_chapter_template = r"""
# 你是一位专业的文学编辑。请仔细阅读我提供的【章节全文】。
# 你的任务是提取结构化信息，并 **必须** 调用 `extract_chapter_details` 函数来返回结果。
# 请严格按照函数参数的描述（特别是关于“情节摘要导语”的详细程度要求）来填充信息。
# **绝对不要** 输出任何 JSON 格式之外的文本、解释、代码块标记（如 ```json ... ```）或 Markdown。
# 直接调用函数并填充其参数。

# 【章节全文】：
# {chapter_text}
# """

# # —— 分析单章（接受 summary_description，包含 JSON 修复） ——
# def analyze_chapter(
#     path: Path,
#     prompt_template: str,
#     summary_description: str, # 具体的摘要要求
#     retries: int = 3
# ) -> str | None:
#     """
#     使用 Gemini 分析单个小说章节文件，强制调用函数并返回 JSON 字符串。
#     允许通过 summary_description 指定摘要的详细程度。
#     如果分析失败，则返回 None。
#     """
#     text = _auto_decode(path)
#     if not text:
#         logging.error(f"无法读取或解码文件: {path.name}")
#         return None

#     full_prompt = prompt_template.format(chapter_text=text)

#     # --- 动态构建函数声明 ---
#     current_schema = copy.deepcopy(base_json_schema)
#     current_schema["properties"]["情节摘要导语"]["description"] = summary_description
#     extract_details_func_declaration = {
#         "name": "extract_chapter_details",
#         "description": "提取小说章节的结构化信息，包括情节摘要、人物、道具、场景和伏笔。",
#         "parameters": current_schema
#     }
#     # --- 动态构建结束 ---

#     safety_settings = {
#         HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
#         HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
#         HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
#         HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
#     }
#     # 注意：如果 API Key 无效，此处也可能抛出异常，尽管 configure 时已检查
#     try:
#         gemini_model = genai.GenerativeModel(
#             GEMINI_MODEL,
#             safety_settings=safety_settings
#         )
#     except Exception as model_init_err:
#         logging.error(f"初始化 Gemini 模型时出错: {model_init_err}", exc_info=True)
#         return None # 无法初始化模型，直接失败

#     # --- 重试循环 ---
#     for attempt in range(retries):
#         func_call_args_raw = None # 用于在出错时记录原始参数
#         try:
#             logging.info(f"开始分析章节: {path.name} (摘要要求: '{summary_description}', 尝试 {attempt + 1}/{retries})")
#             rsp = gemini_model.generate_content(
#                 full_prompt,
#                 generation_config={"temperature": 0.3},
#                 tools=[{"function_declarations": [extract_details_func_declaration]}],
#                 tool_config={'function_calling_config': 'ANY'}
#             )

#             # --- 健壮性检查 ---
#             if not rsp.candidates:
#                  logging.warning(f"分析失败 (尝试 {attempt + 1}): API 未返回候选内容。响应: {rsp}")
#                  time.sleep(2 ** attempt + random.uniform(0, 1))
#                  continue

#             first_candidate = rsp.candidates[0]
#             if not first_candidate.content or not first_candidate.content.parts:
#                 # 检查是否有 block 是因为安全设置
#                 if first_candidate.finish_reason == genai.types.FinishReason.SAFETY:
#                      logging.warning(f"分析失败 (尝试 {attempt + 1}): 内容被安全设置阻止。安全评级: {first_candidate.safety_ratings}")
#                 else:
#                      logging.warning(f"分析失败 (尝试 {attempt + 1}): 候选内容为空或无 Part。完成原因: {first_candidate.finish_reason}")
#                 time.sleep(2 ** attempt + random.uniform(0, 1))
#                 continue

#             # --- 查找函数调用 ---
#             func_call_part = None
#             for part in first_candidate.content.parts:
#                 if part.function_call:
#                     func_call_part = part
#                     break

#             if func_call_part and func_call_part.function_call:
#                 fc = func_call_part.function_call
#                 func_call_args_raw = fc.args # 保存原始参数以备调试

#                 if fc.name == "extract_chapter_details":
#                     logging.info(f"成功分析章节: {path.name} (摘要要求: '{summary_description}')")

#                   # --- FIX v2: Handle Nested RepeatedComposite ---
#                     try:
#                         # 1. Convert top-level MapComposite to dict (shallow)
#                         args_dict_shallow = dict(fc.args)

#                         # 2. Create a new dict for fully native Python types
#                         args_dict_native = {}
#                         for key, value in args_dict_shallow.items():
#                             # Check if the value is the problematic list type
#                             # Using type().__name__ is slightly brittle but targets the known issue.
#                             # isinstance() might be better if we knew the exact class path.
#                             if type(value).__name__ == 'RepeatedComposite':
#                                 # Convert RepeatedComposite to a standard Python list
#                                 # This assumes the items *within* the list are already native types
#                                 # (like strings, which matches your schema: "items": {"type": "string"})
#                                 args_dict_native[key] = list(value)
#                             else:
#                                 # Assume other types (like the string for '情节摘要导语') are already native or serializable
#                                 args_dict_native[key] = value

#                         # 3. Now serialize the deeply converted dict
#                         json_output = json.dumps(args_dict_native, ensure_ascii=False, indent=2)
#                         return json_output # Success

#                     except TypeError as json_err:
#                         # Catch potential errors during the deeper conversion or final serialization
#                         logging.error(f"序列化从API接收的参数时出错: {json_err}", exc_info=False)
#                         logging.error(f"  未能序列化的原始参数类型: {type(fc.args)}")
#                         # Log the shallow dict as well, as it might show the structure better
#                         logging.error(f"  浅层转换后的字典内容 (部分): {str(args_dict_shallow)[:500]}...")
#                         # Continue to the next retry attempt
#                         time.sleep(2 ** attempt + random.uniform(0, 1))
#                         continue # Important: go to next retry if serialization fails
#                     # --- END FIX v2 ---
#                     # --- END FIX ---

#                 else:
#                     logging.warning(f"分析警告 (尝试 {attempt + 1}): 模型调用了意外的函数 '{fc.name}'")
#             else:
#                 # 模型未调用函数
#                 finish_reason = first_candidate.finish_reason
#                 safety_ratings = first_candidate.safety_ratings
#                 logging.warning(f"分析失败 (尝试 {attempt + 1}): 未找到预期的函数调用。")
#                 logging.warning(f"  完成原因: {finish_reason}")
#                 if safety_ratings: # 仅当存在时打印
#                     logging.warning(f"  安全评级: {safety_ratings}")
#                 # 如果有文本输出，也记录下来帮助调试
#                 text_output = ""
#                 try:
#                     text_output = first_candidate.text
#                 except ValueError: # 有时访问 .text 会出错如果内容不是文本
#                      if first_candidate.content and first_candidate.content.parts:
#                          text_output = str(first_candidate.content.parts[0]) # 尝试获取原始部分
#                 if text_output:
#                      logging.warning(f"  模型返回内容 (部分): {text_output[:200]}...") # 只记录部分文本

#             # 如果代码执行到这里，表示当前尝试失败，准备下一次重试
#             time.sleep(2 ** attempt + random.uniform(0, 1))

#         except Exception as e:
#             # 捕获所有其他在 API 调用或处理期间的异常
#             error_context = ""
#             if func_call_args_raw is not None: # 如果出错前已获取参数
#                  error_context = f" | 参数类型: {type(func_call_args_raw)}, 内容 (部分): {str(func_call_args_raw)[:200]}..."

#             # 检查是否是特定的 API 错误类型 (可选，需要 import)
#             # from google.api_core.exceptions import GoogleAPIError
#             # if isinstance(e, GoogleAPIError): ...

#             logging.error(f"Gemini API 调用或处理时发生异常 (尝试 {attempt + 1}): {e}{error_context}", exc_info=True) # 记录完整堆栈
#             time.sleep(2 ** attempt + random.uniform(0, 1)) # 等待后重试

#     # --- 重试结束 ---
#     logging.error(f"❌ 分析失败，已达最大重试次数: {path.name} (摘要要求: '{summary_description}')")
#     return None # 所有重试失败后返回 None


# # —— 批量处理目录下所有章节 (接受 summary_description) ——
# def run_analysis(
#     chapter_root: str,
#     prompt_template: str,
#     summary_description: str, # 摘要要求
#     out_dir: str,
#     mode: tuple[int, int] = (1, 150) # 处理章节范围
# ):
#     """
#     批量处理指定目录下的所有小说章节。
#     """
#     root = Path(chapter_root)
#     if not root.is_dir():
#         logging.error(f"错误：输入目录 '{chapter_root}' 不存在或不是一个目录。")
#         return

#     out_base = Path(out_dir)
#     out_base.mkdir(parents=True, exist_ok=True)
#     logging.info(f"开始批量分析 (摘要要求: '{summary_description}'), 输出到: {out_base}")

#     books = [d for d in root.iterdir() if d.is_dir()]
#     if not books:
#         logging.warning(f"在 '{chapter_root}' 下未找到任何小说子目录。")
#         return

#     s, e = mode
#     logging.info(f"处理章节范围: {s} 到 {e}")

#     total_processed = 0
#     total_failed = 0
#     total_skipped = 0

#     for book in tqdm(books, desc="📚 处理小说书目"):
#         # 查找符合命名规范（以3位数字开头）且在范围内的 txt 文件
#         chapters = sorted([
#             p for p in book.glob("*.txt")
#             if p.name[:3].isdigit() and s <= int(p.name[:3]) <= e
#         ], key=lambda p: int(p.name[:3])) # 按数字排序

#         if not chapters:
#             logging.warning(f"在 '{book.name}' 目录中未找到符合范围 {s}-{e} 的章节文件 (例如 '001_xxx.txt')。")
#             continue

#         out_book_dir = out_base / book.name
#         out_book_dir.mkdir(parents=True, exist_ok=True)

#         processed_count = 0
#         failed_count = 0
#         skipped_count = 0

#         for chap_path in tqdm(chapters, desc=f"📖 分析 '{book.name}'", leave=False):
#             # 输出文件名使用原始章节名 + '_analysis.json'
#             out_filename = f"{chap_path.stem}_analysis.json"
#             out_path = out_book_dir / out_filename

#             if out_path.exists():
#                 skipped_count += 1
#                 continue # 如果已存在，则跳过

#             try:
#                 # 调用分析函数，传入 summary_description
#                 result_json = analyze_chapter(chap_path, prompt_template, summary_description)

#                 if result_json:
#                     out_path.write_text(result_json, encoding="utf-8")
#                     processed_count += 1
#                 else:
#                     # analyze_chapter 内部已记录错误，这里只计数
#                     failed_count += 1
#             except Exception as e:
#                 # 捕获 analyze_chapter 未处理的意外错误
#                 logging.error(f"处理章节 {chap_path.name} 时发生意外错误: {e}", exc_info=True)
#                 failed_count += 1

#         logging.info(f"完成处理 '{book.name}': {processed_count} 个成功, {failed_count} 个失败, {skipped_count} 个跳过。")
#         total_processed += processed_count
#         total_failed += failed_count
#         total_skipped += skipped_count

#     print(f"\n✅ 所有章节结构化分析完成（或尝试完成）。")
#     print(f"  总计: {total_processed} 个成功, {total_failed} 个失败, {total_skipped} 个跳过。")
#     print(f"  结果保存在对应的输出子目录中，根目录为：{OUTPUT_BASE_DIR}") # 指向总目录


# # --- 主程序入口 ---
# if __name__ == "__main__":
#     # 定义输入目录 (★★★ 根据你的实际情况修改 ★★★)
#     CHAPTERS_INPUT_DIR = "/content/novels_chapters"
#     # 定义输出根目录 (★★★ 根据你的实际情况修改 ★★★)
#     OUTPUT_BASE_DIR = "/content/json_results/json_gemini"

#     # --- 定义不同摘要长度的要求 ---
#     summary_req_short = "生成一个非常简短的核心情节摘要（ 约 150-200 字）"
#     summary_req_medium = "生成一个标准的情节摘要，概括主要内容（约 300-400 字）"
#     summary_req_long = "生成一个比较详细的情节摘要，包含更多细节和转折（约 600-700 字）"

#     # --- 运行不同版本的分析 ---
#     # (你可以取消注释掉不想运行的版本)

#     logging.info("="*20 + " 开始简短摘要分析 " + "="*20)
#     run_analysis(
#         chapter_root=CHAPTERS_INPUT_DIR,
#         prompt_template=prompt_chapter_template,
#         summary_description=summary_req_short, # <---- 传入简短要求
#         out_dir=os.path.join(OUTPUT_BASE_DIR, "50000_summary"), # 输出到子目录
#         mode=(1, 150) # 分析 1 到 150 章
#     )

#     logging.info("="*20 + " 开始标准摘要分析 " + "="*20)
#     run_analysis(
#         chapter_root=CHAPTERS_INPUT_DIR,
#         prompt_template=prompt_chapter_template,
#         summary_description=summary_req_medium, # <---- 传入标准要求
#         out_dir=os.path.join(OUTPUT_BASE_DIR, "100000_summary"), # 输出到子目录
#         mode=(1, 150)
#     )

#     logging.info("="*20 + " 开始详细摘要分析 " + "="*20)
#     run_analysis(
#         chapter_root=CHAPTERS_INPUT_DIR,
#         prompt_template=prompt_chapter_template,
#         summary_description=summary_req_long, # <---- 传入详细要求
#         out_dir=os.path.join(OUTPUT_BASE_DIR, "200000_summary"), # 输出到子目录
#         mode=(1, 150)
#     )

#     logging.info("="*20 + " 所有不同摘要长度的分析任务已提交 " + "="*20)

In [7]:
# # =============================
# # ✅ Gemini 分析小说章节，结构化输出 JSON + TXT细纲生成 (主程序参数更新)
# # =============================
# # !pip install -q --upgrade google-generativeai chardet tqdm
# # !pip install -U google-generativeai # 确保安装最新版本

# import os
# import json
# import re
# import time
# import random
# import string
# import chardet
# import logging
# import copy
# from pathlib import Path
# from typing import List, Dict, Any, Tuple
# from tqdm.auto import tqdm
# import google.generativeai as genai
# from google.generativeai.types import HarmCategory, HarmBlockThreshold

# # --- (API 初始化, 通用函数, Schema, Prompt 模板 - 与上一版本相同) ---

# # —— API 初始化 (⚠️ 警告：直接写入 API 密钥极不安全！) ——
# api_key = "your-default-api-key" # ⚠️ 替换为您的真实 API 密钥
# if not api_key or api_key == "YOUR_API_KEY_HERE":
#     raise ValueError("❌ 错误：请务必将代码中的 'YOUR_API_KEY_HERE' 替换为您的真实 Gemini API 密钥。")
# try:
#     genai.configure(api_key=api_key)
#     logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
#     logging.info("Gemini API 已使用直接写入的密钥进行配置。")
# except Exception as e:
#     print(f"CRITICAL: 使用提供的 API 密钥配置 Gemini SDK 时出错: {e}")
#     raise ValueError(f"API 密钥配置失败: {e}")

# GEMINI_MODEL = "gemini-2.0-flash-latest"

# def _auto_decode(path: Path) -> str:
#     try:
#         raw = path.read_bytes()
#         enc = chardet.detect(raw)["encoding"] or "utf-8"
#         if enc.lower() not in ['utf-8', 'gbk', 'gb2312', 'big5']:
#              try: return raw.decode('utf-8', errors='ignore').strip()
#              except UnicodeDecodeError:
#                  try: return raw.decode('gbk', errors='ignore').strip()
#                  except UnicodeDecodeError: return raw.decode(enc, errors='ignore').strip()
#         return raw.decode(enc, errors="ignore").strip()
#     except Exception as e:
#         logging.error(f"读取文件 {path.name} 时出错: {e}")
#         return ""

# def _rand_tag(k=6): return ''.join(random.choices(string.ascii_uppercase, k=k))

# base_json_schema = {
#     "type": "object",
#     "properties": {
#         "情节摘要导语": {"type": "string", "description": ""},
#         "出现人物": {"type": "array", "items": {"type": "string"}, "description": "本章出现的所有人物名称列表"},
#         "出现道具": {"type": "array", "items": {"type": "string"}, "description": "本章出现的关键道具列表"},
#         "出现场景": {"type": "array", "items": {"type": "string"}, "description": "本章发生故事的主要场景列表"},
#         "伏笔_设下": {"type": "array", "items": {"type": "string"}, "description": "本章新埋下的伏笔描述"},
#         "伏笔_回收": {"type": "array", "items": {"type": "string"}, "description": "本章回收或呼应的过往伏笔描述"}
#     },
#     "required": ["情节摘要导语", "出现人物", "出现道具", "出现场景", "伏笔_设下", "伏笔_回收"]
# }

# prompt_chapter_template = r"""
# 你是一位专业的文学编辑。请仔细阅读我提供的【章节全文】。
# 你的任务是提取结构化信息，并 **必须** 调用 `extract_chapter_details` 函数来返回结果。
# 请严格按照函数参数的描述（特别是关于“情节摘要导语”的详细程度要求）来填充信息。
# **绝对不要** 输出任何 JSON 格式之外的文本、解释、代码块标记（如 ```json ... ```）或 Markdown。
# 直接调用函数并填充其参数。

# 【章节全文】：
# {chapter_text}
# """

# # --- (analyze_chapter 函数 - 与上一版本修复后相同) ---
# def analyze_chapter(
#     path: Path,
#     prompt_template: str,
#     summary_description: str,
#     retries: int = 3
# ) -> str | None:
#     text = _auto_decode(path)
#     if not text: return None
#     full_prompt = prompt_template.format(chapter_text=text)
#     current_schema = copy.deepcopy(base_json_schema)
#     current_schema["properties"]["情节摘要导语"]["description"] = summary_description
#     extract_details_func_declaration = {
#         "name": "extract_chapter_details",
#         "description": "提取小说章节的结构化信息...",
#         "parameters": current_schema
#     }
#     safety_settings = { HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE, HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE, HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE, HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE, }
#     try:
#         gemini_model = genai.GenerativeModel(GEMINI_MODEL, safety_settings=safety_settings)
#     except Exception as model_init_err:
#         logging.error(f"初始化 Gemini 模型时出错: {model_init_err}", exc_info=True)
#         return None

#     for attempt in range(retries):
#         func_call_args_raw = None
#         try:
#             logging.info(f"开始分析章节: {path.name} (摘要要求: '{summary_description}', 尝试 {attempt + 1}/{retries})")
#             rsp = gemini_model.generate_content(full_prompt, generation_config={"temperature": 0.3}, tools=[{"function_declarations": [extract_details_func_declaration]}], tool_config={'function_calling_config': 'ANY'})
#             if not rsp.candidates: logging.warning(f"分析失败 (尝试 {attempt + 1}): API 未返回候选内容。"); time.sleep(2 ** attempt + random.uniform(0, 1)); continue
#             first_candidate = rsp.candidates[0]
#             if not first_candidate.content or not first_candidate.content.parts:
#                 if first_candidate.finish_reason == genai.types.FinishReason.SAFETY: logging.warning(f"分析失败 (尝试 {attempt + 1}): 内容被安全设置阻止。")
#                 else: logging.warning(f"分析失败 (尝试 {attempt + 1}): 候选内容为空。原因: {first_candidate.finish_reason}")
#                 time.sleep(2 ** attempt + random.uniform(0, 1)); continue
#             func_call_part = None
#             for part in first_candidate.content.parts:
#                 if part.function_call: func_call_part = part; break
#             if func_call_part and func_call_part.function_call:
#                 fc = func_call_part.function_call
#                 func_call_args_raw = fc.args
#                 if fc.name == "extract_chapter_details":
#                     logging.info(f"成功分析章节: {path.name} (摘要要求: '{summary_description}')")
#                     try:
#                         args_dict_shallow = dict(fc.args)
#                         args_dict_native = {}
#                         for key, value in args_dict_shallow.items():
#                             if type(value).__name__ == 'RepeatedComposite':
#                                 args_dict_native[key] = list(value)
#                             else:
#                                 args_dict_native[key] = value
#                         json_output = json.dumps(args_dict_native, ensure_ascii=False, indent=2)
#                         return json_output
#                     except TypeError as json_err:
#                         logging.error(f"序列化参数时出错: {json_err}", exc_info=False)
#                         logging.error(f"  原始参数类型: {type(func_call_args_raw)}")
#                         logging.error(f"  浅层字典 (部分): {str(args_dict_shallow)[:500]}...")
#                         time.sleep(2 ** attempt + random.uniform(0, 1)); continue # Go to next retry on serialization error
#                 else: logging.warning(f"分析警告 (尝试 {attempt + 1}): 调用了意外函数 '{fc.name}'")
#             else:
#                 finish_reason = first_candidate.finish_reason; safety_ratings = first_candidate.safety_ratings
#                 logging.warning(f"分析失败 (尝试 {attempt + 1}): 未找到函数调用。原因: {finish_reason}")
#                 if safety_ratings: logging.warning(f"  安全评级: {safety_ratings}")
#                 try: text_output = first_candidate.text
#                 except ValueError: text_output = str(first_candidate.content.parts[0]) if first_candidate.content and first_candidate.content.parts else ""
#                 if text_output: logging.warning(f"  模型返回内容 (部分): {text_output[:200]}...")
#             time.sleep(2 ** attempt + random.uniform(0, 1)) # Wait before next retry if this attempt failed here
#         except Exception as e:
#             error_context = f" | 参数类型: {type(func_call_args_raw)}, 内容 (部分): {str(func_call_args_raw)[:200]}..." if func_call_args_raw else ""
#             logging.error(f"API 调用或处理时发生异常 (尝试 {attempt + 1}): {e}{error_context}", exc_info=True)
#             time.sleep(2 ** attempt + random.uniform(0, 1)) # Wait before next retry on general exception
#     logging.error(f"❌ 分析失败，已达最大重试次数: {path.name} (摘要要求: '{summary_description}')")
#     return None


# # --- (run_analysis 函数 - 与上一版本相同) ---
# def run_analysis(chapter_root: str, prompt_template: str, summary_description: str, out_dir: str, mode: tuple[int, int] = (1, 150)):
#     root = Path(chapter_root); out_base = Path(out_dir)
#     if not root.is_dir(): logging.error(f"错误：输入目录 '{chapter_root}' 不存在。"); return
#     out_base.mkdir(parents=True, exist_ok=True)
#     logging.info(f"开始批量分析 (摘要要求: '{summary_description}'), 输出到: {out_base}")
#     books = [d for d in root.iterdir() if d.is_dir()]
#     if not books: logging.warning(f"在 '{chapter_root}' 下未找到任何小说子目录。"); return
#     s, e = mode; logging.info(f"处理章节范围: {s} 到 {e}")
#     total_processed, total_failed, total_skipped = 0, 0, 0
#     for book in tqdm(books, desc="📚 处理小说书目"):
#         chapters = sorted([p for p in book.glob("*.txt") if p.name[:3].isdigit() and s <= int(p.name[:3]) <= e], key=lambda p: int(p.name[:3]))
#         if not chapters: logging.warning(f"在 '{book.name}' 目录中未找到符合范围 {s}-{e} 的章节文件。"); continue
#         out_book_dir = out_base / book.name; out_book_dir.mkdir(parents=True, exist_ok=True)
#         processed_count, failed_count, skipped_count = 0, 0, 0
#         for chap_path in tqdm(chapters, desc=f"📖 分析 '{book.name}'", leave=False):
#             out_filename = f"{chap_path.stem}_analysis.json"; out_path = out_book_dir / out_filename
#             if out_path.exists(): skipped_count += 1; continue
#             try:
#                 result_json = analyze_chapter(chap_path, prompt_template, summary_description)
#                 if result_json: out_path.write_text(result_json, encoding="utf-8"); processed_count += 1
#                 else: failed_count += 1
#             except Exception as e: logging.error(f"处理章节 {chap_path.name} 时发生意外错误: {e}", exc_info=True); failed_count += 1
#         logging.info(f"完成处理 '{book.name}': {processed_count} 成功, {failed_count} 失败, {skipped_count} 跳过。")
#         total_processed += processed_count; total_failed += failed_count; total_skipped += skipped_count
#     # Ensure the final message regarding where results are saved uses the correct base directory variable
#     print(f"\n✅ JSON 分析完成。总计: {total_processed} 成功, {total_failed} 失败, {total_skipped} 跳过。")
#     print(f"  JSON 结果保存在: {out_dir}") # Use out_dir specific to this run_analysis call


# # --- (JSON 转 TXT 及合并功能 - format_list_output, convert_json_to_txt, merge_txt_outlines, run_post_processing - 与上一版本相同) ---

# def format_list_output(items: List[str]) -> str:
#     """Helper function to format lists for TXT output."""
#     if not items: return "- 无"
#     return "\n".join(f"- {item}" for item in items)

# def convert_json_to_txt(json_path: Path, txt_path: Path) -> bool:
#     """Reads JSON analysis file, writes formatted TXT outline."""
#     try:
#         with open(json_path, 'r', encoding='utf-8') as f: data = json.load(f)
#         chapter_title = json_path.stem.replace("_analysis", "")
#         output_lines = [
#             f"章节：{chapter_title}", "\n【情节摘要导语】", data.get("情节摘要导语", "N/A"),
#             "\n【出现人物】", format_list_output(data.get("出现人物", [])),
#             "\n【出现道具】", format_list_output(data.get("出现道具", [])),
#             "\n【出现场景】", format_list_output(data.get("出现场景", [])),
#             "\n【伏笔_设下】", format_list_output(data.get("伏笔_设下", [])),
#             "\n【伏笔_回收】", format_list_output(data.get("伏笔_回收", [])),
#             "\n" + "-" * 40 + "\n"
#         ]
#         txt_path.parent.mkdir(parents=True, exist_ok=True)
#         with open(txt_path, 'w', encoding='utf-8') as f: f.write("\n".join(output_lines))
#         return True
#     except FileNotFoundError: logging.error(f"JSON 文件未找到: {json_path}"); return False
#     except json.JSONDecodeError: logging.error(f"无法解析 JSON 文件: {json_path}"); return False
#     except Exception as e: logging.error(f"转换 {json_path.name} 到 TXT 时发生意外错误: {e}", exc_info=True); return False

# def merge_txt_outlines(txt_dir: Path, output_file: Path) -> bool:
#     """Merges all chapter TXT outlines in a directory into a single file."""
#     try:
#         chapter_files = list(txt_dir.glob("*_outline.txt"))
#         if not chapter_files: logging.warning(f"在目录 {txt_dir} 中未找到要合并的 TXT 文件 (*_outline.txt)。"); return False
#         def get_chapter_num(file_path: Path) -> int:
#             try: match = re.match(r"(\d+)", file_path.name); return int(match.group(1)) if match else float('inf')
#             except ValueError: return float('inf')
#         chapter_files.sort(key=get_chapter_num)
#         merged_content = []
#         logging.info(f"开始合并 {len(chapter_files)} 个 TXT 文件到 {output_file.name}...")
#         for chap_file in tqdm(chapter_files, desc=f"  合并 TXT", leave=False):
#             try:
#                 with open(chap_file, 'r', encoding='utf-8') as f: merged_content.append(f.read())
#             except Exception as e: logging.error(f"读取 TXT 文件 {chap_file.name} 时出错: {e}")
#         if not merged_content: logging.error(f"未能读取任何 TXT 文件内容进行合并。"); return False
#         output_file.parent.mkdir(parents=True, exist_ok=True)
#         with open(output_file, 'w', encoding='utf-8') as f: f.write("".join(merged_content))
#         logging.info(f"成功合并 TXT 文件到: {output_file}")
#         return True
#     except Exception as e: logging.error(f"合并 TXT 文件到 {output_file.name} 时发生意外错误: {e}", exc_info=True); return False

# def run_post_processing(base_json_dir: str):
#     """Runs JSON-to-TXT conversion and merging for all books in a base directory."""
#     base_path = Path(base_json_dir)
#     if not base_path.is_dir(): logging.warning(f"跳过后期处理：目录 {base_path} 不存在。"); return
#     logging.info(f"\n{'='*20} 开始对 '{base_path.name}' 进行后期处理 (TXT 生成与合并) {'='*20}")
#     book_dirs = [d for d in base_path.iterdir() if d.is_dir()]
#     if not book_dirs: logging.warning(f"在 {base_path} 中未找到书籍子目录进行后期处理。"); return
#     total_books_processed, total_books_failed = 0, 0
#     for book_json_dir in tqdm(book_dirs, desc="📚 处理书籍 (后期)"):
#         book_name = book_json_dir.name
#         book_txt_outlines_dir = book_json_dir / f"{book_name}_txt_outlines"
#         merged_output_file = base_path / f"{book_name}_完整细纲.txt" # Save merged file one level up
#         logging.info(f"处理书籍 '{book_name}': JSON={book_json_dir}, TXT={book_txt_outlines_dir}, Merged={merged_output_file}")
#         json_files = list(book_json_dir.glob("*_analysis.json"))
#         if not json_files: logging.warning(f"  在 {book_json_dir} 中未找到 JSON 文件进行转换。"); continue
#         conversion_success_count, conversion_fail_count = 0, 0
#         logging.info(f"  开始转换 {len(json_files)} 个 JSON 文件到 TXT...")
#         for json_file in tqdm(json_files, desc=f"  转换 JSON", leave=False):
#             txt_filename = json_file.stem.replace("_analysis", "_outline.txt")
#             txt_file_path = book_txt_outlines_dir / txt_filename
#             if convert_json_to_txt(json_file, txt_file_path): conversion_success_count += 1
#             else: conversion_fail_count += 1
#         logging.info(f"  JSON 到 TXT 转换完成: {conversion_success_count} 成功, {conversion_fail_count} 失败。")
#         if conversion_success_count == 0:
#             logging.error(f"  未能成功转换任何 JSON 文件为 TXT，跳过合并步骤。"); total_books_failed += 1; continue
#         if merge_txt_outlines(book_txt_outlines_dir, merged_output_file): total_books_processed += 1
#         else: total_books_failed +=1; logging.error(f"  未能成功合并 '{book_name}' 的 TXT 细纲。")
#     logging.info(f"\n{'='*20} '{base_path.name}' 后期处理完成 {'='*20}")
#     logging.info(f"  成功生成完整细纲的书籍数量: {total_books_processed}")
#     logging.info(f"  处理失败或未完成的书籍数量: {total_books_failed}")


INFO: Gemini API 已使用直接写入的密钥进行配置。


In [ ]:
# # ========================================
# # ==           主程序入口 (更新)        ==
# # ========================================
# if __name__ == "__main__":
#     # 定义输入目录 (★★★ 根据你的实际情况修改 ★★★)
#     CHAPTERS_INPUT_DIR = "/content/novels_normalized"
#     # 定义输出根目录 (★★★ 根据你的实际情况修改 ★★★)
#     OUTPUT_BASE_DIR = "/content/json_results/json_gemini_normalized" # <--- 更新

#     # --- 定义不同摘要长度的要求 (更新) ---
#     summary_req_short = "生成一个非常简短的核心情节摘要（ 约 150-200 字）" # <--- 更新
#     summary_req_medium = "生成一个标准的情节摘要，概括主要内容（约 300-400 字）" # <--- 更新
#     summary_req_long = "生成一个比较详细的情节摘要，包含更多细节和转折（约 600-700 字）" # <--- 更新

#     # --- 运行不同版本的分析 ---
#     # (你可以取消注释掉不想运行的版本)
#     analysis_dirs_to_process = [] # Store dirs for post-processing

#     logging.info("="*20 + " 开始简短摘要分析 (150-200字) " + "="*20)
#     short_summary_dir = os.path.join(OUTPUT_BASE_DIR, "30000_summary") # <--- 更新
#     run_analysis(
#         chapter_root=CHAPTERS_INPUT_DIR,
#         prompt_template=prompt_chapter_template,
#         summary_description=summary_req_short, # 使用更新后的描述
#         out_dir=short_summary_dir,             # 使用更新后的目录
#         mode=(1, 200) # 分析 1 到 150 章
#     )
#     analysis_dirs_to_process.append(short_summary_dir) # 添加更新后的目录

#     logging.info("="*20 + " 开始标准摘要分析 (300-400字) " + "="*20)
#     medium_summary_dir = os.path.join(OUTPUT_BASE_DIR, "50000_summary") # <--- 更新
#     run_analysis(
#         chapter_root=CHAPTERS_INPUT_DIR,
#         prompt_template=prompt_chapter_template,
#         summary_description=summary_req_medium, # 使用更新后的描述
#         out_dir=medium_summary_dir,             # 使用更新后的目录
#         mode=(1, 200)
#     )
#     analysis_dirs_to_process.append(medium_summary_dir) # 添加更新后的目录

#     logging.info("="*20 + " 开始详细摘要分析 (600-700字) " + "="*20)
#     long_summary_dir = os.path.join(OUTPUT_BASE_DIR, "100000_summary") # <--- 更新
#     run_analysis(
#         chapter_root=CHAPTERS_INPUT_DIR,
#         prompt_template=prompt_chapter_template,
#         summary_description=summary_req_long, # 使用更新后的描述
#         out_dir=long_summary_dir,             # 使用更新后的目录
#         mode=(1, 200)
#     )
#     analysis_dirs_to_process.append(long_summary_dir) # 添加更新后的目录

#     logging.info("\n" + "="*20 + " 所有 JSON 分析任务已完成/提交 " + "="*20)

#     # --- 运行后期处理：生成 TXT 细纲 ---
#     # (这部分保持不变，它会使用上面 analysis_dirs_to_process 列表中的新目录)
#     logging.info("\n" + "="*20 + " 开始运行后期处理 (生成 TXT 细纲) " + "="*20)
#     for dir_to_process in analysis_dirs_to_process:
#         run_post_processing(dir_to_process)

#     logging.info("\n" + "="*20 + " 全部处理流程结束 " + "="*20)

In [12]:
!rm -rf /content/json_results/json_gemini_normalized

In [2]:
# # ===============================================================
# # 一键把 *_analysis.json → 每本书一个《完整细纲.txt》
# # 支持文件名形态：
# #   118_第二十八章xxx_analysis.json
# #   第005章_xxx_analysis.json
# #   书名_001_analysis.json（也OK）
# #   ——核心：文件名里**第一个数字**认为是章号，
# #   书名直接取最外层文件夹名。
# # ===============================================================

# #!pip install -q tqdm pandas          # ← 如本环境已装可注释

# from pathlib import Path
# import re, json, tqdm, logging, pandas as pd

# # ================= 修改这里 =================
# JSON_ROOT    = Path("/content/json_results/json_gemini_normalized/30000_summary")   # 放 JSON 的根目录
# TXT_OUT_ROOT = Path("/content/json_results/json_gemini_normalized/30000_summary_gathered")                # 输出根目录
# # ============================================
# TXT_OUT_ROOT.mkdir(parents=True, exist_ok=True)

# logging.basicConfig(level=logging.INFO,
#                     format="%(asctime)s - %(levelname)s - %(message)s")

# # ---------- 简单工具 ----------
# DIGIT_RE = re.compile(r"(\d{1,4})")          # 抓 1-4 位章号

# def pick_book_and_chapter(path: Path):
#     """返回 (book_id, chap_no)；识别失败 → None"""
#     book_id = path.parent.name               # 父文件夹名即书名
#     m = DIGIT_RE.match(path.stem) or DIGIT_RE.search(path.stem)
#     if not m:
#         return None
#     return book_id, int(m.group(1))

# def bullets(lst):
#     return "- 无" if not lst else "\n".join(f"- {x}" for x in lst)

# def json_to_outline(jp: Path) -> str:
#     data = json.loads(jp.read_text(encoding="utf-8"))
#     chap_tag = jp.stem.replace("_analysis", "")
#     return "\n".join([
#         f"章节：{chap_tag}",
#         "\n【情节摘要导语】",
#         data.get("情节摘要导语","N/A"),
#         "\n【出现人物】",
#         bullets(data.get("出现人物", [])),
#         "\n【出现道具】",
#         bullets(data.get("出现道具", [])),
#         "\n【出现场景】",
#         bullets(data.get("出现场景", [])),
#         "\n【伏笔_设下】",
#         bullets(data.get("伏笔_设下", [])),
#         "\n【伏笔_回收】",
#         bullets(data.get("伏笔_回收", [])),
#         "\n" + "-"*40 + "\n"
#     ])

# # ---------- 把所有 JSON 归簇 ----------
# book_files = {}
# for jp in JSON_ROOT.rglob("*_analysis.json"):
#     res = pick_book_and_chapter(jp)
#     if not res:
#         logging.warning(f"跳过无法识别文件名: {jp}")
#         continue
#     book, chap = res
#     book_files.setdefault(book, []).append((chap, jp))

# if not book_files:
#     raise SystemExit(f"❌ 在 {JSON_ROOT} 下没找到 *_analysis.json")

# # ---------- 逐书写 TXT ----------
# stats = []
# for book, lst in tqdm.tqdm(book_files.items(), desc="📚 处理书籍"):
#     lst.sort(key=lambda x: x[0])                     # 按章号
#     out_dir = TXT_OUT_ROOT / book
#     out_dir.mkdir(parents=True, exist_ok=True)

#     merged, ok, bad = [], 0, 0
#     for num, jp in lst:
#         try:
#             outline = json_to_outline(jp)
#             (out_dir / f"{num:03}_outline.txt").write_text(outline, "utf-8")
#             merged.append(outline); ok += 1
#         except Exception as e:
#             logging.error(f"解析 {jp} 出错: {e}"); bad += 1

#     (out_dir / "完整细纲.txt").write_text("".join(merged), "utf-8")
#     stats.append(dict(book=book, chapters=len(lst), ok=ok, bad=bad,
#                       out_file=str(out_dir / '完整细纲.txt')))

# # ---------- 汇总显示 ----------
# df = pd.DataFrame(stats)
# print("\n✅ 全部完成。概览：")
# print(df.to_string(index=False))
# print(f"\n所有 TXT 已输出至：{TXT_OUT_ROOT.resolve()}")

📚 处理书籍: 100%|██████████| 40/40 [00:00<00:00, 45.87it/s]


✅ 全部完成。概览：
                           book  chapters  ok  bad                                                                                          out_file
    《奋斗在新明朝》(校对版全本)作者_随轻风去_utf8       149 149    0     /content/json_results/json_gemini/30000_summary_gathered/《奋斗在新明朝》(校对版全本)作者_随轻风去_utf8/完整细纲.txt
   《反正我是超能力者》(校对版全本)作者_吃书妖_utf8       150 150    0    /content/json_results/json_gemini/30000_summary_gathered/《反正我是超能力者》(校对版全本)作者_吃书妖_utf8/完整细纲.txt
        《天可汗》(校对版全本)作者_西风紧_utf8       150 150    0         /content/json_results/json_gemini/30000_summary_gathered/《天可汗》(校对版全本)作者_西风紧_utf8/完整细纲.txt
《崩坏世界的传奇大冒险》(精校版全本)作者_国王陛下_utf8       145 145    0 /content/json_results/json_gemini/30000_summary_gathered/《崩坏世界的传奇大冒险》(精校版全本)作者_国王陛下_utf8/完整细纲.txt
        《全球进化》(精校版全本)作者_咬狗_utf8       150 150    0         /content/json_results/json_gemini/30000_summary_gathered/《全球进化》(精校版全本)作者_咬狗_utf8/完整细纲.txt
      《武林半侠传》(校对版全本)作者_文抄公_utf8       150 150    0       /content/json_results/json_gemini/300

In [50]:
#汇总细纲
from pathlib import Path
import re, json, tqdm, logging

# ================= 修改这里 =================
JSON_ROOT    = Path("/content/json_results/json_gemini_normalized/50000_summary")   # JSON 输入目录
TXT_OUT_ROOT = Path("/content/json_results/json_gemini_normalized/100000_summary_gathered")  # 输出目录
# ============================================
TXT_OUT_ROOT.mkdir(parents=True, exist_ok=True)

logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s - %(levelname)s - %(message)s")

DIGIT_RE = re.compile(r"(\d{1,4})")

def pick_book_and_chapter(path: Path):
    book_id = path.parent.name
    m = DIGIT_RE.match(path.stem) or DIGIT_RE.search(path.stem)
    if not m:
        return None
    return book_id, int(m.group(1))

def bullets(lst):
    return "- 无" if not lst else "\n".join(f"- {x}" for x in lst)

def json_to_outline(jp: Path) -> str:
    data = json.loads(jp.read_text(encoding="utf-8"))
    chap_tag = jp.stem.replace("_analysis", "")

    # 加入移除“第n章”模式的正则表达式
    chapter_pattern = re.compile(r'(第\s*[零一二三四五六七八九十百千万亿\d]+\s*[章节回集篇部])|([零一二三四五六七八九十百千万亿\d]+\s*[章节回集篇部])')    
    summary_intro = chapter_pattern.sub('', data.get("情节摘要导语", "N/A"))

    return "\n".join([
        f"章节：{chap_tag}",
        "\n【情节摘要导语】",
        summary_intro.strip(),
        "\n【出现人物】",
        bullets(data.get("出现人物", [])),
        "\n【出现道具】",
        bullets(data.get("出现道具", [])),
        "\n【出现场景】",
        bullets(data.get("出现场景", [])),
        "\n【伏笔_设下】",
        bullets(data.get("伏笔_设下", [])),
        "\n【伏笔_回收】",
        bullets(data.get("伏笔_回收", [])),
        "\n" + "-" * 40 + "\n"
    ])

# ---------- 聚合 JSON ----------
book_files = {}
for jp in JSON_ROOT.rglob("*_analysis.json"):
    res = pick_book_and_chapter(jp)
    if not res:
        logging.warning(f"跳过无法识别文件名: {jp}")
        continue
    book, chap = res
    book_files.setdefault(book, []).append((chap, jp))

if not book_files:
    raise SystemExit(f"❌ 在 {JSON_ROOT} 下没找到 *_analysis.json")

# ---------- 汇总并写入 ----------
for book, lst in tqdm.tqdm(book_files.items(), desc="📚 处理书籍"):
    lst.sort(key=lambda x: x[0])
    merged, ok, bad = [], 0, 0
    for _, jp in lst:
        try:
            outline = json_to_outline(jp)
            merged.append(outline)
            ok += 1
        except Exception as e:
            logging.error(f"解析 {jp} 出错: {e}")
            bad += 1

    full_text = "".join(merged)
    print(f"\n📘 【{book}】 完整细纲（共 {ok} 章）")
    print("=" * 60)
    #print(full_text)
    print("=" * 60 + "\n")

    out_dir = TXT_OUT_ROOT / book
    out_dir.mkdir(parents=True, exist_ok=True)
    out_file = out_dir / "entire_section_outline.txt"
    out_file.write_text(full_text, encoding="utf-8")

📚 处理书籍:  38%|███▊      | 15/40 [00:00<00:00, 141.43it/s]


📘 【《奋斗在新明朝》（校对版全本）作者：随轻风去_utf8】 完整细纲（共 199 章）


📘 【《反正我是超能力者》（校对版全本）作者：吃书妖_utf8】 完整细纲（共 199 章）


📘 【《天可汗》（校对版全本）作者：西风紧_utf8】 完整细纲（共 198 章）


📘 【《崩坏世界的传奇大冒险》（精校版全本）作者：国王陛下_utf8】 完整细纲（共 198 章）


📘 【《全球进化》（精校版全本）作者：咬狗_utf8】 完整细纲（共 197 章）


📘 【《武林半侠传》（校对版全本）作者：文抄公_utf8】 完整细纲（共 200 章）


📘 【国宴大厨在八零】 完整细纲（共 200 章）


📘 【《搜神记》（精校版全本）作者：树下野狐_utf8】 完整细纲（共 199 章）


📘 【重生八零：毒妻不好惹】 完整细纲（共 200 章）


📘 【《窃明》（校对版全本）作者：大爆炸(灰熊猫)_utf8】 完整细纲（共 200 章）


📘 【《蜀山》（精校版全本）作者：流浪的蛤蟆_utf8】 完整细纲（共 200 章）


📘 【《陈二狗的妖孽人生》（校对版全本）作者：烽火戏诸侯_utf8】 完整细纲（共 200 章）


📘 【《贩罪》（精校版全本）作者：三天两觉_utf8】 完整细纲（共 200 章）


📘 【《重生之出人头地》（校对版全本）作者：闹闹不爱闹_utf8】 完整细纲（共 197 章）


📘 【八零喜事：当家肥妻大翻身】 完整细纲（共 199 章）


📘 【《肆虐韩娱》（校对版全本）作者：姬叉_utf8】 完整细纲（共 197 章）


📘 【八零年代好时光】 完整细纲（共 200 章）


📘 【《食物链顶端的男人》（校对版全本）作者：熊狼狗_utf8】 完整细纲（共 199 章）


📘 【《高手寂寞2》（校对版全本）作者：兰帝魅晨_utf8】 完整细纲（共 197 章）


📘 【《黑龙法典》（校对版全本）作者：欢声_utf8】 完整细纲（共 200 章）


📘 【《诛仙》（校对版全本）作者：萧鼎_utf8】 完整细纲（共 200 章）


📘 【《回到过去变成猫》（精校版全本）作者：陈词懒调_utf8】 完整细纲（共 198 章）


📘 【《神游》（校对版全本）作者：徐公子胜治_utf8】 完整细纲（共 19

📚 处理书籍: 100%|██████████| 40/40 [00:00<00:00, 137.50it/s]


📘 【《重活了》（精校版全本）作者： 尝谕_utf8】 完整细纲（共 198 章）


📘 【《雅骚》（校对版全本）作者：贼道三痴_utf8】 完整细纲（共 198 章）


📘 【八零福星俏媳妇】 完整细纲（共 200 章）


📘 【《上品寒士》（校对版全本）作者：贼道三痴_utf8】 完整细纲（共 200 章）


📘 【《十州风云志》（校对版全本）作者：知秋_utf8】 完整细纲（共 198 章）


📘 【《史上第一混乱》（校对版全本）作者：张小花_utf8】 完整细纲（共 198 章）


📘 【《随波逐流之一代军师》（校对版全本）作者：随波逐流_utf8】 完整细纲（共 199 章）


📘 【重生八零：佳妻致富忙】 完整细纲（共 197 章）


📘 【重回八零过好日子】 完整细纲（共 200 章）


📘 【《绝对一番》（校对版全本）作者：海底漫步者_utf8】 完整细纲（共 198 章）


📘 【《我的女友是恶女》（校对版全本）作者：海底漫步者_utf8】 完整细纲（共 195 章）



In [ ]:
# # =============================
# # ✅ Gemini 分析小说章节，结构化输出 JSON + TXT细纲生成 (主程序参数更新)
# # =============================
# # !pip install -q --upgrade google-generativeai chardet tqdm
# # !pip install -U google-generativeai # 确保安装最新版本

# import os
# import json
# import re
# import time
# import random
# import string
# import chardet
# import logging
# import copy
# from pathlib import Path
# from typing import List, Dict, Any, Tuple
# from tqdm.auto import tqdm
# import google.generativeai as genai
# from google.generativeai.types import HarmCategory, HarmBlockThreshold

# # --- (API 初始化, 通用函数, Schema, Prompt 模板 - 与上一版本相同) ---

# # —— API 初始化 (⚠️ 警告：直接写入 API 密钥极不安全！) ——
# api_key = "your-default-api-key" # ⚠️ 替换为您的真实 API 密钥
# if not api_key or api_key == "YOUR_API_KEY_HERE":
#     raise ValueError("❌ 错误：请务必将代码中的 'YOUR_API_KEY_HERE' 替换为您的真实 Gemini API 密钥。")
# try:
#     genai.configure(api_key=api_key)
#     logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
#     logging.info("Gemini API 已使用直接写入的密钥进行配置。")
# except Exception as e:
#     print(f"CRITICAL: 使用提供的 API 密钥配置 Gemini SDK 时出错: {e}")
#     raise ValueError(f"API 密钥配置失败: {e}")

# GEMINI_MODEL = "gemini-2.0-flash-latest"

# def _auto_decode(path: Path) -> str:
#     try:
#         raw = path.read_bytes()
#         enc = chardet.detect(raw)["encoding"] or "utf-8"
#         if enc.lower() not in ['utf-8', 'gbk', 'gb2312', 'big5']:
#              try: return raw.decode('utf-8', errors='ignore').strip()
#              except UnicodeDecodeError:
#                  try: return raw.decode('gbk', errors='ignore').strip()
#                  except UnicodeDecodeError: return raw.decode(enc, errors='ignore').strip()
#         return raw.decode(enc, errors="ignore").strip()
#     except Exception as e:
#         logging.error(f"读取文件 {path.name} 时出错: {e}")
#         return ""

# def _rand_tag(k=6): return ''.join(random.choices(string.ascii_uppercase, k=k))

# base_json_schema = {
#     "type": "object",
#     "properties": {
#         "情节摘要导语": {"type": "string", "description": ""},
#         "出现人物": {"type": "array", "items": {"type": "string"}, "description": "本章出现的所有人物名称列表"},
#         "出现道具": {"type": "array", "items": {"type": "string"}, "description": "本章出现的关键道具列表"},
#         "出现场景": {"type": "array", "items": {"type": "string"}, "description": "本章发生故事的主要场景列表"},
#         "伏笔_设下": {"type": "array", "items": {"type": "string"}, "description": "本章新埋下的伏笔描述"},
#         "伏笔_回收": {"type": "array", "items": {"type": "string"}, "description": "本章回收或呼应的过往伏笔描述"}
#     },
#     "required": ["情节摘要导语", "出现人物", "出现道具", "出现场景", "伏笔_设下", "伏笔_回收"]
# }

# prompt_chapter_template = r"""
# 你是一位专业的文学编辑。请仔细阅读我提供的【章节全文】。
# 你的任务是提取结构化信息，并 **必须** 调用 `extract_chapter_details` 函数来返回结果。
# 请严格按照函数参数的描述（特别是关于“情节摘要导语”的详细程度要求）来填充信息。
# **绝对不要** 输出任何 JSON 格式之外的文本、解释、代码块标记（如 ```json ... ```）或 Markdown。
# 直接调用函数并填充其参数。

# 【章节全文】：
# {chapter_text}
# """

# # --- (analyze_chapter 函数 - 与上一版本修复后相同) ---
# def analyze_chapter(
#     path: Path,
#     prompt_template: str,
#     summary_description: str,
#     retries: int = 3
# ) -> str | None:
#     text = _auto_decode(path)
#     if not text: return None
#     full_prompt = prompt_template.format(chapter_text=text)
#     current_schema = copy.deepcopy(base_json_schema)
#     current_schema["properties"]["情节摘要导语"]["description"] = summary_description
#     extract_details_func_declaration = {
#         "name": "extract_chapter_details",
#         "description": "提取小说章节的结构化信息...",
#         "parameters": current_schema
#     }
#     safety_settings = { HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE, HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE, HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE, HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE, }
#     try:
#         gemini_model = genai.GenerativeModel(GEMINI_MODEL, safety_settings=safety_settings)
#     except Exception as model_init_err:
#         logging.error(f"初始化 Gemini 模型时出错: {model_init_err}", exc_info=True)
#         return None

#     for attempt in range(retries):
#         func_call_args_raw = None
#         try:
#             logging.info(f"开始分析章节: {path.name} (摘要要求: '{summary_description}', 尝试 {attempt + 1}/{retries})")
#             rsp = gemini_model.generate_content(full_prompt, generation_config={"temperature": 0.3}, tools=[{"function_declarations": [extract_details_func_declaration]}], tool_config={'function_calling_config': 'ANY'})
#             if not rsp.candidates: logging.warning(f"分析失败 (尝试 {attempt + 1}): API 未返回候选内容。"); time.sleep(2 ** attempt + random.uniform(0, 1)); continue
#             first_candidate = rsp.candidates[0]
#             if not first_candidate.content or not first_candidate.content.parts:
#                 if first_candidate.finish_reason == genai.types.FinishReason.SAFETY: logging.warning(f"分析失败 (尝试 {attempt + 1}): 内容被安全设置阻止。")
#                 else: logging.warning(f"分析失败 (尝试 {attempt + 1}): 候选内容为空。原因: {first_candidate.finish_reason}")
#                 time.sleep(2 ** attempt + random.uniform(0, 1)); continue
#             func_call_part = None
#             for part in first_candidate.content.parts:
#                 if part.function_call: func_call_part = part; break
#             if func_call_part and func_call_part.function_call:
#                 fc = func_call_part.function_call
#                 func_call_args_raw = fc.args
#                 if fc.name == "extract_chapter_details":
#                     logging.info(f"成功分析章节: {path.name} (摘要要求: '{summary_description}')")
#                     try:
#                         args_dict_shallow = dict(fc.args)
#                         args_dict_native = {}
#                         for key, value in args_dict_shallow.items():
#                             if type(value).__name__ == 'RepeatedComposite':
#                                 args_dict_native[key] = list(value)
#                             else:
#                                 args_dict_native[key] = value
#                         json_output = json.dumps(args_dict_native, ensure_ascii=False, indent=2)
#                         return json_output
#                     except TypeError as json_err:
#                         logging.error(f"序列化参数时出错: {json_err}", exc_info=False)
#                         logging.error(f"  原始参数类型: {type(func_call_args_raw)}")
#                         logging.error(f"  浅层字典 (部分): {str(args_dict_shallow)[:500]}...")
#                         time.sleep(2 ** attempt + random.uniform(0, 1)); continue # Go to next retry on serialization error
#                 else: logging.warning(f"分析警告 (尝试 {attempt + 1}): 调用了意外函数 '{fc.name}'")
#             else:
#                 finish_reason = first_candidate.finish_reason; safety_ratings = first_candidate.safety_ratings
#                 logging.warning(f"分析失败 (尝试 {attempt + 1}): 未找到函数调用。原因: {finish_reason}")
#                 if safety_ratings: logging.warning(f"  安全评级: {safety_ratings}")
#                 try: text_output = first_candidate.text
#                 except ValueError: text_output = str(first_candidate.content.parts[0]) if first_candidate.content and first_candidate.content.parts else ""
#                 if text_output: logging.warning(f"  模型返回内容 (部分): {text_output[:200]}...")
#             time.sleep(2 ** attempt + random.uniform(0, 1)) # Wait before next retry if this attempt failed here
#         except Exception as e:
#             error_context = f" | 参数类型: {type(func_call_args_raw)}, 内容 (部分): {str(func_call_args_raw)[:200]}..." if func_call_args_raw else ""
#             logging.error(f"API 调用或处理时发生异常 (尝试 {attempt + 1}): {e}{error_context}", exc_info=True)
#             time.sleep(2 ** attempt + random.uniform(0, 1)) # Wait before next retry on general exception
#     logging.error(f"❌ 分析失败，已达最大重试次数: {path.name} (摘要要求: '{summary_description}')")
#     return None


# # --- (run_analysis 函数 - 与上一版本相同) ---
# def run_analysis(chapter_root: str, prompt_template: str, summary_description: str, out_dir: str, mode: tuple[int, int] = (1, 150)):
#     root = Path(chapter_root); out_base = Path(out_dir)
#     if not root.is_dir(): logging.error(f"错误：输入目录 '{chapter_root}' 不存在。"); return
#     out_base.mkdir(parents=True, exist_ok=True)
#     logging.info(f"开始批量分析 (摘要要求: '{summary_description}'), 输出到: {out_base}")
#     books = [d for d in root.iterdir() if d.is_dir()]
#     if not books: logging.warning(f"在 '{chapter_root}' 下未找到任何小说子目录。"); return
#     s, e = mode; logging.info(f"处理章节范围: {s} 到 {e}")
#     total_processed, total_failed, total_skipped = 0, 0, 0
#     for book in tqdm(books, desc="📚 处理小说书目"):
#         chapters = sorted([p for p in book.glob("*.txt") if p.name[:3].isdigit() and s <= int(p.name[:3]) <= e], key=lambda p: int(p.name[:3]))
#         if not chapters: logging.warning(f"在 '{book.name}' 目录中未找到符合范围 {s}-{e} 的章节文件。"); continue
#         out_book_dir = out_base / book.name; out_book_dir.mkdir(parents=True, exist_ok=True)
#         processed_count, failed_count, skipped_count = 0, 0, 0
#         for chap_path in tqdm(chapters, desc=f"📖 分析 '{book.name}'", leave=False):
#             out_filename = f"{chap_path.stem}_analysis.json"; out_path = out_book_dir / out_filename
#             if out_path.exists(): skipped_count += 1; continue
#             try:
#                 result_json = analyze_chapter(chap_path, prompt_template, summary_description)
#                 if result_json: out_path.write_text(result_json, encoding="utf-8"); processed_count += 1
#                 else: failed_count += 1
#             except Exception as e: logging.error(f"处理章节 {chap_path.name} 时发生意外错误: {e}", exc_info=True); failed_count += 1
#         logging.info(f"完成处理 '{book.name}': {processed_count} 成功, {failed_count} 失败, {skipped_count} 跳过。")
#         total_processed += processed_count; total_failed += failed_count; total_skipped += skipped_count
#     # Ensure the final message regarding where results are saved uses the correct base directory variable
#     print(f"\n✅ JSON 分析完成。总计: {total_processed} 成功, {total_failed} 失败, {total_skipped} 跳过。")
#     print(f"  JSON 结果保存在: {out_dir}") # Use out_dir specific to this run_analysis call


# # --- (JSON 转 TXT 及合并功能 - format_list_output, convert_json_to_txt, merge_txt_outlines, run_post_processing - 与上一版本相同) ---

# def format_list_output(items: List[str]) -> str:
#     """Helper function to format lists for TXT output."""
#     if not items: return "- 无"
#     return "\n".join(f"- {item}" for item in items)

# def convert_json_to_txt(json_path: Path, txt_path: Path) -> bool:
#     """Reads JSON analysis file, writes formatted TXT outline."""
#     try:
#         with open(json_path, 'r', encoding='utf-8') as f: data = json.load(f)
#         chapter_title = json_path.stem.replace("_analysis", "")
#         output_lines = [
#             f"章节：{chapter_title}", "\n【情节摘要导语】", data.get("情节摘要导语", "N/A"),
#             "\n【出现人物】", format_list_output(data.get("出现人物", [])),
#             "\n【出现道具】", format_list_output(data.get("出现道具", [])),
#             "\n【出现场景】", format_list_output(data.get("出现场景", [])),
#             "\n【伏笔_设下】", format_list_output(data.get("伏笔_设下", [])),
#             "\n【伏笔_回收】", format_list_output(data.get("伏笔_回收", [])),
#             "\n" + "-" * 40 + "\n"
#         ]
#         txt_path.parent.mkdir(parents=True, exist_ok=True)
#         with open(txt_path, 'w', encoding='utf-8') as f: f.write("\n".join(output_lines))
#         return True
#     except FileNotFoundError: logging.error(f"JSON 文件未找到: {json_path}"); return False
#     except json.JSONDecodeError: logging.error(f"无法解析 JSON 文件: {json_path}"); return False
#     except Exception as e: logging.error(f"转换 {json_path.name} 到 TXT 时发生意外错误: {e}", exc_info=True); return False

# def merge_txt_outlines(txt_dir: Path, output_file: Path) -> bool:
#     """Merges all chapter TXT outlines in a directory into a single file."""
#     try:
#         chapter_files = list(txt_dir.glob("*_outline.txt"))
#         if not chapter_files: logging.warning(f"在目录 {txt_dir} 中未找到要合并的 TXT 文件 (*_outline.txt)。"); return False
#         def get_chapter_num(file_path: Path) -> int:
#             try: match = re.match(r"(\d+)", file_path.name); return int(match.group(1)) if match else float('inf')
#             except ValueError: return float('inf')
#         chapter_files.sort(key=get_chapter_num)
#         merged_content = []
#         logging.info(f"开始合并 {len(chapter_files)} 个 TXT 文件到 {output_file.name}...")
#         for chap_file in tqdm(chapter_files, desc=f"  合并 TXT", leave=False):
#             try:
#                 with open(chap_file, 'r', encoding='utf-8') as f: merged_content.append(f.read())
#             except Exception as e: logging.error(f"读取 TXT 文件 {chap_file.name} 时出错: {e}")
#         if not merged_content: logging.error(f"未能读取任何 TXT 文件内容进行合并。"); return False
#         output_file.parent.mkdir(parents=True, exist_ok=True)
#         with open(output_file, 'w', encoding='utf-8') as f: f.write("".join(merged_content))
#         logging.info(f"成功合并 TXT 文件到: {output_file}")
#         return True
#     except Exception as e: logging.error(f"合并 TXT 文件到 {output_file.name} 时发生意外错误: {e}", exc_info=True); return False

# def run_post_processing(base_json_dir: str):
#     """Runs JSON-to-TXT conversion and merging for all books in a base directory."""
#     base_path = Path(base_json_dir)
#     if not base_path.is_dir(): logging.warning(f"跳过后期处理：目录 {base_path} 不存在。"); return
#     logging.info(f"\n{'='*20} 开始对 '{base_path.name}' 进行后期处理 (TXT 生成与合并) {'='*20}")
#     book_dirs = [d for d in base_path.iterdir() if d.is_dir()]
#     if not book_dirs: logging.warning(f"在 {base_path} 中未找到书籍子目录进行后期处理。"); return
#     total_books_processed, total_books_failed = 0, 0
#     for book_json_dir in tqdm(book_dirs, desc="📚 处理书籍 (后期)"):
#         book_name = book_json_dir.name
#         book_txt_outlines_dir = book_json_dir / f"{book_name}_txt_outlines"
#         merged_output_file = base_path / f"{book_name}_完整细纲.txt" # Save merged file one level up
#         logging.info(f"处理书籍 '{book_name}': JSON={book_json_dir}, TXT={book_txt_outlines_dir}, Merged={merged_output_file}")
#         json_files = list(book_json_dir.glob("*_analysis.json"))
#         if not json_files: logging.warning(f"  在 {book_json_dir} 中未找到 JSON 文件进行转换。"); continue
#         conversion_success_count, conversion_fail_count = 0, 0
#         logging.info(f"  开始转换 {len(json_files)} 个 JSON 文件到 TXT...")
#         for json_file in tqdm(json_files, desc=f"  转换 JSON", leave=False):
#             txt_filename = json_file.stem.replace("_analysis", "_outline.txt")
#             txt_file_path = book_txt_outlines_dir / txt_filename
#             if convert_json_to_txt(json_file, txt_file_path): conversion_success_count += 1
#             else: conversion_fail_count += 1
#         logging.info(f"  JSON 到 TXT 转换完成: {conversion_success_count} 成功, {conversion_fail_count} 失败。")
#         if conversion_success_count == 0:
#             logging.error(f"  未能成功转换任何 JSON 文件为 TXT，跳过合并步骤。"); total_books_failed += 1; continue
#         if merge_txt_outlines(book_txt_outlines_dir, merged_output_file): total_books_processed += 1
#         else: total_books_failed +=1; logging.error(f"  未能成功合并 '{book_name}' 的 TXT 细纲。")
#     logging.info(f"\n{'='*20} '{base_path.name}' 后期处理完成 {'='*20}")
#     logging.info(f"  成功生成完整细纲的书籍数量: {total_books_processed}")
#     logging.info(f"  处理失败或未完成的书籍数量: {total_books_failed}")


In [ ]:
# # ========================================
# # ==           主程序入口 (更新)        ==
# # ========================================
# if __name__ == "__main__":
#     # 定义输入目录 (★★★ 根据你的实际情况修改 ★★★)
#     CHAPTERS_INPUT_DIR = "/content/novels_normalized"
#     # 定义输出根目录 (★★★ 根据你的实际情况修改 ★★★)
#     OUTPUT_BASE_DIR = "/content/json_results/json_gemini_normalized" # <--- 更新

#     # --- 定义不同摘要长度的要求 (更新) ---
#     summary_req_short = "生成一个非常简短的核心情节摘要（ 约 150-200 字）" # <--- 更新
#     summary_req_medium = "生成一个标准的情节摘要，概括主要内容（约 300-400 字）" # <--- 更新
#     summary_req_long = "生成一个比较详细的情节摘要，包含更多细节和转折（约 600-700 字）" # <--- 更新

#     # --- 运行不同版本的分析 ---
#     # (你可以取消注释掉不想运行的版本)
#     analysis_dirs_to_process = [] # Store dirs for post-processing

#     logging.info("="*20 + " 开始简短摘要分析 (150-200字) " + "="*20)
#     short_summary_dir = os.path.join(OUTPUT_BASE_DIR, "30000_summary") # <--- 更新
#     run_analysis(
#         chapter_root=CHAPTERS_INPUT_DIR,
#         prompt_template=prompt_chapter_template,
#         summary_description=summary_req_short, # 使用更新后的描述
#         out_dir=short_summary_dir,             # 使用更新后的目录
#         mode=(1, 200) # 分析 1 到 150 章
#     )
#     analysis_dirs_to_process.append(short_summary_dir) # 添加更新后的目录

#     logging.info("="*20 + " 开始标准摘要分析 (300-400字) " + "="*20)
#     medium_summary_dir = os.path.join(OUTPUT_BASE_DIR, "50000_summary") # <--- 更新
#     run_analysis(
#         chapter_root=CHAPTERS_INPUT_DIR,
#         prompt_template=prompt_chapter_template,
#         summary_description=summary_req_medium, # 使用更新后的描述
#         out_dir=medium_summary_dir,             # 使用更新后的目录
#         mode=(1, 200)
#     )
#     analysis_dirs_to_process.append(medium_summary_dir) # 添加更新后的目录

#     logging.info("="*20 + " 开始详细摘要分析 (600-700字) " + "="*20)
#     long_summary_dir = os.path.join(OUTPUT_BASE_DIR, "100000_summary") # <--- 更新
#     run_analysis(
#         chapter_root=CHAPTERS_INPUT_DIR,
#         prompt_template=prompt_chapter_template,
#         summary_description=summary_req_long, # 使用更新后的描述
#         out_dir=long_summary_dir,             # 使用更新后的目录
#         mode=(1, 200)
#     )
#     analysis_dirs_to_process.append(long_summary_dir) # 添加更新后的目录

#     logging.info("\n" + "="*20 + " 所有 JSON 分析任务已完成/提交 " + "="*20)

#     # --- 运行后期处理：生成 TXT 细纲 ---
#     # (这部分保持不变，它会使用上面 analysis_dirs_to_process 列表中的新目录)
#     logging.info("\n" + "="*20 + " 开始运行后期处理 (生成 TXT 细纲) " + "="*20)
#     for dir_to_process in analysis_dirs_to_process:
#         run_post_processing(dir_to_process)

#     logging.info("\n" + "="*20 + " 全部处理流程结束 " + "="*20)

In [13]:
# # ===============================================================
# # ① 依赖
# # ===============================================================
# !pip install -q --upgrade google-generativeai tqdm pandas chardet

# # ===============================================================
# # ② 路径 & KEY
# # ===============================================================
# from pathlib import Path
# import google.generativeai as genai
# from google.generativeai.types import HarmCategory, HarmBlockThreshold
# import json, logging, random, time, textwrap, tqdm, pandas as pd

# TXT_SRC = Path("/content/json_results/json_gemini/30000_summary_gathered")
# DIR_1000   = Path("/content/json_results/json_gemini/1000_global_json")
# DIR_5000   = Path("/content/json_results/json_gemini/5000_global_json")
# DIR_10000  = Path("/content/json_results/json_gemini/10000_global_json")
# for d in (DIR_1000, DIR_5000, DIR_10000): d.mkdir(parents=True, exist_ok=True)

# API_KEY = "your-default-api-key"
# GEMINI_MODEL = "gemini-1.5-flash"      # 同章节脚本风格

# genai.configure(api_key=API_KEY)
# logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# SAFETY_SETTINGS = {
#     HarmCategory.HARM_CATEGORY_HARASSMENT       : HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
#     HarmCategory.HARM_CATEGORY_HATE_SPEECH      : HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
#     HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
#     HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
# }

# # ===============================================================
# # ③ prompt 模板 & schemas
# # ===============================================================
# BASE_SCHEMA = {
#     "type":"object",
#     "properties":{
#         "total_characters":{"type":"array","items":{"type":"string"}},
#         "total_scenes"    :{"type":"array","items":{"type":"string"}},
#         "total_items"     :{"type":"array","items":{"type":"string"}},
#         "summary"         :{"type":"string"}
#     },
#     "required":["total_characters","total_scenes","total_items","summary"]
# }

# FUNC_DECL = {
#     "name":"extract_book_outline",
#     "description":"整书大纲提炼：去重三张列表 + 指定字数剧情大纲",
#     "parameters":BASE_SCHEMA
# }

# PROMPT_TMPL = textwrap.dedent("""
# 你是一位严谨的出版编辑。阅读《完整细纲》后：
# 1. 去重列出全书【人物】【场景】【道具】。
# 2. 写一篇约 {target_len} 字（±10%）的剧情大纲。
# 必须调用 extract_book_outline 函数返回 JSON，仅含
#  total_characters / total_scenes / total_items / summary
# 三个列表与一段大纲。

# 《完整细纲》：
# {outline}
# """)

# # ===============================================================
# # ④ Gemini 调用（与章节脚本同风格）
# # ===============================================================
# def call_gemini(prompt: str, func_decl: dict, retries=3):
#     gm = genai.GenerativeModel(GEMINI_MODEL, safety_settings=SAFETY_SETTINGS)
#     for i in range(retries):
#         try:
#             rsp = gm.generate_content(
#                 prompt,
#                 generation_config={"temperature":0.3},
#                 tools=[{"function_declarations":[func_decl]}],
#                 tool_config={"function_calling_config":"ANY"}
#             )
#             cand = rsp.candidates[0]
#             # --- 解析 function_call ----
#             func_part = next((p for p in cand.content.parts if p.function_call), None)
#             if func_part and func_part.function_call and \
#                func_part.function_call.name == "extract_book_outline":
#                 # protobuf → python
#                 def to_py(v):
#                     if type(v).__name__ == "RepeatedComposite": return list(v)
#                     return v
#                 return {k:to_py(v) for k,v in dict(func_part.function_call.args).items()}
#             # --- 备用：普通 text 返回 ---
#             if cand.content.parts and cand.content.parts[0].text:
#                 return json.loads(cand.content.parts[0].text)
#             if hasattr(cand,"text") and cand.text:
#                 return json.loads(cand.text)
#             logging.warning(f"Gemini 无有效输出（尝试{i+1}） finish_reason={cand.finish_reason}")
#         except Exception as e:
#             logging.error(f"Gemini 调用异常（尝试{i+1}）：{e}")
#         time.sleep(2**i+random.random())
#     return None

# def save_json(out_dir:Path, book:str, data:dict):
#     (out_dir/f"{book}.json").write_text(json.dumps(data,ensure_ascii=False,indent=2),"utf-8")

# # ===============================================================
# # ⑤ 主循环
# # ===============================================================
# stats=[]
# for outline_path in tqdm.tqdm(TXT_SRC.rglob("*完整细纲*.txt"), desc="📚 处理书籍"):
#     book = outline_path.parent.name
#     if (DIR_1000/f"{book}.json").exists() and (DIR_5000/f"{book}.json").exists() and (DIR_10000/f"{book}.json").exists():
#         logging.info(f"跳过《{book}》—三档已存在"); continue

#     outline = outline_path.read_text(encoding="utf-8")[:250_000]  # 控制长度
#     for tag, tgt, out_dir in [("1000",1000,DIR_1000), ("5000",5000,DIR_5000), ("10000",10000,DIR_10000)]:
#         out_file = out_dir / f"{book}.json"
#         if out_file.exists(): continue

#         prompt = PROMPT_TMPL.format(target_len=tgt, outline=outline)
#         result = call_gemini(prompt, FUNC_DECL)
#         if result:
#             # schema里是统一字段名 summary → 重新 key
#             common = {k: result[k] for k in ("total_characters","total_scenes","total_items")}
#             save_json(out_dir, book, {**common, f"summary_{tag}": result["summary"]})
#             logging.info(f"生成 {tag} 字大纲成功：《{book}》")
#         else:
#             logging.error(f"生成 {tag} 字大纲失败：《{book}》")
#     stats.append(dict(book=book))

# # ===============================================================
# # ⑥ 结果概览
# # ===============================================================
# print("\n完成，目录：")
# print("1000字 ⇒", DIR_1000)
# print("5000字 ⇒", DIR_5000)
# print("10000字 ⇒", DIR_10000)

📚 处理书籍: 0it [00:52, ?it/s]


KeyboardInterrupt: 

In [ ]:
# ===============================================================
# ✅ Gemini 总纲结构化调用（参考章节脚本，完全兼容）
# ===============================================================
!pip install -q --upgrade google-generativeai tqdm chardet

import os, json, logging, random, time, textwrap
from pathlib import Path
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
from tqdm import tqdm

# ===============================================================
# ① 设置参数
# ===============================================================
API_KEY = "your-default-api-key"  # ← 替换
GEMINI_MODEL = "gemini-2.0-flash"  # 推荐用 1.5 处理整书，flash 容易爆上下文
BOOK_DIR = Path("/content/json_results/json_gemini/30000_summary_gathered/《贩罪》(精校版全本)作者_三天两觉_utf8")
OUTLINE_FILE = BOOK_DIR / "完整细纲.txt"
TARGET_LEN = 1000  # 可设 5000 或 10000

# ===============================================================
# ② API 初始化（不加 safety）
# ===============================================================
genai.configure(api_key=API_KEY)
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
model = genai.GenerativeModel(GEMINI_MODEL)

# ===============================================================
# ③ Function Schema + Prompt
# ===============================================================
base_schema = {
    "type": "object",
    "properties": {
        "total_characters": {"type": "array", "items": {"type": "string"}},
        "total_scenes": {"type": "array", "items": {"type": "string"}},
        "total_items": {"type": "array", "items": {"type": "string"}},
        "summary": {"type": "string"}
    },
    "required": ["total_characters", "total_scenes", "total_items", "summary"]
}

function_declaration = {
    "name": "extract_book_outline",
    "description": "提炼整本小说的核心结构信息与剧情大纲",
    "parameters": base_schema
}

prompt_template = textwrap.dedent(f"""
你是一位专业出版编辑。请阅读下面提供的《完整细纲》：

任务：
1. 去重列出【人物】【场景】【道具】三类信息。
2. 写出一篇 ≈ {TARGET_LEN} 字（±10%）的剧情大纲。

必须调用 extract_book_outline 函数，返回 JSON 格式。
返回字段固定为：
  - total_characters
  - total_scenes
  - total_items
  - summary

《完整细纲》：
{{chapter_text}}
""")

# ===============================================================
# ④ 调用函数（完全照章节版风格）
# ===============================================================
def analyze_outline_text(text: str, retries: int = 3) -> dict | None:
    full_prompt = prompt_template.format(chapter_text=text)
    print(f"\n📝 Prompt 预览（前500字）：\n{textwrap.shorten(full_prompt, width=500)}\n")

    for attempt in range(retries):
        try:
            logging.info(f"🔁 Gemini 第 {attempt + 1}/{retries} 次调用中...")
            rsp = model.generate_content(
                full_prompt,
                generation_config={"temperature": 0.3},
                tools=[{"function_declarations": [function_declaration]}],
                tool_config={'function_calling_config': 'ANY'}
            )

            candidate = rsp.candidates[0]
            logging.info(f"✅ 接收响应，finish_reason: {candidate.finish_reason}")

            if hasattr(candidate, "safety_ratings"):
                logging.info(f"🔒 安全评级: {[r.category.name for r in candidate.safety_ratings]}")

            parts = candidate.content.parts
            print(f"📦 返回内容 parts 数量：{len(parts)}")

            for i, part in enumerate(parts):
                print(f"🔍 Part[{i}] 类型：", 
                      "function_call" if part.function_call else "text", 
                      " / 内容预览:", str(part)[:120].replace("\n", " "))

                # -- 主函数解析 --
                if part.function_call and part.function_call.name == "extract_book_outline":
                    logging.info(f"✅ 发现函数调用 extract_book_outline")
                    args = part.function_call.args
                    native = {k: list(v) if hasattr(v, '__iter__') and not isinstance(v, str) else v
                              for k, v in dict(args).items()}
                    return native

            # -- fallback: 纯 JSON 文本 --
            if parts and hasattr(parts[0], "text") and parts[0].text.strip().startswith("{"):
                logging.warning("⚠️ 未使用函数调用，尝试 fallback 为纯 JSON 文本解析")
                return json.loads(parts[0].text)

        except Exception as e:
            logging.error(f"Gemini 调用失败：{e}", exc_info=True)
            time.sleep(2 ** attempt + random.random())

    logging.error("❌ 所有尝试失败，未获取任何结构化数据")
    return None


# ===============================================================
# ⑤ 运行分析
# ===============================================================
if not OUTLINE_FILE.exists():
    raise FileNotFoundError(f"未找到细纲文件：{OUTLINE_FILE}")
text = OUTLINE_FILE.read_text(encoding="utf-8")[:300_000]

result = analyze_outline_text(text)
if result:
    print("✅ 结果字段：", list(result.keys()))
    # print("📌 摘要开头：", result["summary"][:200], "...")
    # 保存
    output_path = Path(f"/content/{BOOK_DIR.name}_summary_{TARGET_LEN}.json")
    output_path.write_text(json.dumps(result, ensure_ascii=False, indent=2))
    print("✅ 已保存至：", output_path)
else:
    print("❌ 未能获得结果")

In [ ]:
# # ============================================================
# # ✅ 20 000 字细纲 → ≤10 000 字大纲（Gemini 2.0 Flash）
# #   · 遍历 /content/json_results/json_gemini/30000_summary_gathered
# #   · 只处理目标细纲：
# #       《上品寒士》(校对版全本)作者_贼道三痴_utf8/完整细纲.txt
# #   · 输出：同目录下生成  full_outline_trimmed.txt
# # ============================================================

# # 0⃣ 依赖
# # ------------------------------------------------------------
# # !pip install -U google-generativeai chardet tqdm

# import os, chardet, logging, time, random, string
# from pathlib import Path
# from tqdm.auto import tqdm
# import google.generativeai as genai

# # 1⃣ API 初始化
# # ------------------------------------------------------------
# os.environ["GEMINI_API_KEY"] = "your-api-key"
# genai.configure(api_key=os.environ["GEMINI_API_KEY"])
# gemini = genai.GenerativeModel("gemini-2.0-flash")

# # 2⃣ 工具函数
# # ------------------------------------------------------------
# def _auto_decode(p: Path) -> str:
#     raw = p.read_bytes()
#     enc = chardet.detect(raw)["encoding"] or "utf-8"
#     return raw.decode(enc, errors="ignore")

# def _rand_tag(k=6):
#     return ''.join(random.choices(string.ascii_uppercase, k=k))

# def summarize_to_10k(text: str,
#                      prompt_header: str,
#                      retries: int = 3) -> str:
#     base_prompt = f"{prompt_header}\n\n#TAG:{_rand_tag()}"
#     for attempt in range(retries):
#         try:
#             rsp = gemini.generate_content(
#                 [base_prompt, text],
#                 generation_config={"temperature": 0.2,
#                                    "max_output_tokens": 8192}
#             )
#             if rsp.text:
#                 return rsp.text.strip()
#         except Exception as e:
#             logging.warning(f"Gemini retry {attempt+1}: {e}")
#             time.sleep(2 ** attempt)
#     raise RuntimeError("Gemini summarization failed")

# # 3⃣ 主流程
# # ------------------------------------------------------------
# ROOT = Path("/content/json_results/json_gemini/30000_summary_gathered")
# target_rel = "《上品寒士》(校对版全本)作者_贼道三痴_utf8/完整细纲.txt"
# target_path = ROOT / target_rel

# if not target_path.exists():
#     raise FileNotFoundError(f"未找到目标文件: {target_path}")

# full_outline = _auto_decode(target_path)
# logging.info(f"原细纲字数: {len(full_outline):,}")

# PROMPT_HEADER = (
#     "你是一位资深中文小说编辑，请将下面约 50 000 字的【整书细纲】"
#     "精炼为总字数 ≤10000 字的大纲。\n"
#     "【输出要求】\n"
#     "• 只用简体中文，以章节：大纲情节的格式输出，不要章节这两个字；\n"
#     "• 保持情节完整，突出主要人物、冲突、转折、结局；\n"
#     "• 不得输出任何额外解释或注释。\n"
#     "• 每个段落要精简，必须涵盖从第一章到最后一章的全部细纲内容。\n\n"
#     "【整书细纲】："
# )

# trimmed_outline = summarize_to_10k(full_outline, PROMPT_HEADER)

# out_path = target_path.with_name("full_outline_trimmed.txt")
# out_path.write_text(trimmed_outline, encoding="utf-8")

# print("✅ 已生成 ≤10 000 字大纲:", out_path)

2025-05-02 17:14:47,984 - INFO - 原细纲字数: 66,242


✅ 已生成 ≤10 000 字大纲: /content/json_results/json_gemini/30000_summary_gathered/《上品寒士》(校对版全本)作者_贼道三痴_utf8/full_outline_trimmed.txt


In [56]:
import os, chardet, logging, time, random, string
from pathlib import Path
from tqdm.auto import tqdm
import google.generativeai as genai

# 初始化 Gemini（只做一次）
def init_gemini(api_key: str):
    os.environ["GEMINI_API_KEY"] = api_key
    genai.configure(api_key=api_key)
    return genai.GenerativeModel("gemini-2.0-flash")

# 解码器
def _auto_decode(p: Path) -> str:
    raw = p.read_bytes()
    enc = chardet.detect(raw)["encoding"] or "utf-8"
    return raw.decode(enc, errors="ignore")

def _rand_tag(k=6):
    return ''.join(random.choices(string.ascii_uppercase, k=k))

# 调用 Gemini API
def summarize(text: str, prompt: str, gemini, retries=3) -> str:
    base_prompt = f"{prompt.strip()}\n\n#TAG:{_rand_tag()}"
    for attempt in range(retries):
        try:
            rsp = gemini.generate_content(
                [base_prompt, text],
                generation_config={"temperature": 0.2, "max_output_tokens": 8192}
            )
            if rsp.text:
                return rsp.text.strip()
        except Exception as e:
            logging.warning(f"Gemini retry {attempt+1}: {e}")
            time.sleep(2 ** attempt)
    raise RuntimeError("Gemini summarization failed")

# ✅ 主函数接口
def summarize_outlines_batch(input_dir: str,
                              input_filename: str,
                              output_dir: str,
                              output_filename: str,
                              prompt_text: str,
                              gemini_api_key: str):
    """
    批量处理细纲，输入输出目录可分离。
    每本书输入路径为 input_dir/书名/input_filename
    每本书输出路径为 output_dir/书名/output_filename
    """
    logging.basicConfig(level=logging.INFO,
                        format="%(asctime)s - %(levelname)s - %(message)s")
    
    gemini = init_gemini(gemini_api_key)
    input_root = Path(input_dir)
    output_root = Path(output_dir)
    output_root.mkdir(parents=True, exist_ok=True)

    book_dirs = [d for d in input_root.iterdir() if d.is_dir()]
    if not book_dirs:
        raise FileNotFoundError(f"❌ No book folders found in {input_root}")

    for book_dir in tqdm(book_dirs, desc="📚 Summarizing Outlines"):
        book_name = book_dir.name
        src = book_dir / input_filename
        dst_dir = output_root / book_name
        dst = dst_dir / output_filename

        if not src.exists():
            logging.warning(f"⚠️ Missing file: {src}")
            continue
        if dst.exists():
            logging.info(f"✅ Output exists, skipping: {dst}")
            continue

        try:
            content = _auto_decode(src)
        except Exception as e:
            logging.error(f"❌ Decode failed for {src}: {e}")
            continue

        try:
            trimmed = summarize(content, prompt_text, gemini)
            dst_dir.mkdir(parents=True, exist_ok=True)
            dst.write_text(trimmed, encoding="utf-8")
            logging.info(f"✅ Written: {dst}")
        except Exception as e:
            logging.error(f"❌ Summarization failed for {book_name}: {e}")

    print(f"\n🎉 All done! Outputs saved to {output_root}")

prompt_10000 = """你是一位资深中文小说编辑，请将下面的【整书细纲】精炼为总字数 ≤10 000 字的大纲。
【输出要求】
• 只用简体中文，以章节：大纲情节的格式输出，不要章节这两个字；
• 保持情节完整，突出主要人物、冲突、转折、结局；
• 不得输出任何额外解释或注释。
• 每个段落要精简，必须涵盖从第一章到最后一章的全部细纲内容。

【整书细纲】：
"""

prompt_10000 = """你是一位资深中文小说编辑，请将下面的【整书细纲】精炼为总字数 ≤10000 字的大纲。
【输出要求】
• 只用简体中文，以第几章节：大纲情节的格式输出，不要章节这两个字，保留章节号；
• 保持情节完整，突出主要人物、冲突、转折、结局；
• 不得输出任何额外解释或注释。
• 每个段落要精简，必须涵盖从第一章到最后一章的全部细纲内容。
不得超过10000字，及每章在25字以内。
【整书细纲】：
"""
prompt_1000 = """你是一位资深中文小说编辑，请将下面的【整书细纲】精炼为总字数 ≤1000 字的大纲。
【输出要求】
• 只用简体中文，以第几章节：大纲情节的格式输出，不要章节这两个字，保留章节号；
• 保持情节完整，突出主要人物、冲突、转折、结局；
• 不得输出任何额外解释或注释。
• 每个段落要精简，必须涵盖从第一章到最后一章的全部细纲内容。
总字数必须不得超过1000字，即每章在5字以内，非常非常简短。200章总结完即停止输出。
【整书细纲】：
"""

prompt_5000 = """你是一位资深中文小说编辑，请将下面的【整书细纲】精炼为总字数 ≤5000 字的大纲。
【输出要求】
• 只用简体中文，以第几章节：大纲情节的格式输出，不要章节这两个字，保留章节号；
• 保持情节完整，突出主要人物、冲突、转折、结局；
• 不得输出任何额外解释或注释。
• 每个段落要精简，必须涵盖从第一章到最后一章的全部细纲内容。
不得超过5000字，即每章在15个字以内。
【整书细纲】：
"""


# summarize_outlines_batch(
#     input_dir="/content/json_results/json_gemini_normalized/30000_summary_gathered",
#     input_filename="entire_section_outline.txt",
#     output_dir="/content/json_results/json_gemini_normalized/30000_outline_10000",
#     output_filename="summary_trimmed.txt",
#     prompt_text=prompt_10000,
#     gemini_api_key="your-default-api-key"
# )

# summarize_outlines_batch(
#     input_dir="/content/json_results/json_gemini_normalized/30000_summary_gathered",
#     input_filename="entire_section_outline.txt",
#     output_dir="/content/json_results/json_gemini_normalized/30000_outline_5000",
#     output_filename="summary_trimmed.txt",
#     prompt_text=prompt_5000,
#     gemini_api_key="your-default-api-key"
# )

summarize_outlines_batch(
    input_dir="/content/json_results/json_gemini_normalized/30000_summary_gathered",
    input_filename="entire_section_outline.txt",
    output_dir="/content/json_results/json_gemini_normalized/30000_outline_1000",
    output_filename="summary_trimmed.txt",
    prompt_text=prompt_1000,
    gemini_api_key="your-default-api-key"
)

📚 Summarizing Outlines:   0%|          | 0/40 [00:00<?, ?it/s]

2025-05-03 14:29:29,023 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/30000_outline_1000/《奋斗在新明朝》（校对版全本）作者：随轻风去_utf8/summary_trimmed.txt
2025-05-03 14:29:42,998 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/30000_outline_1000/《反正我是超能力者》（校对版全本）作者：吃书妖_utf8/summary_trimmed.txt
2025-05-03 14:29:53,509 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/30000_outline_1000/《天可汗》（校对版全本）作者：西风紧_utf8/summary_trimmed.txt
2025-05-03 14:30:07,543 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/30000_outline_1000/《崩坏世界的传奇大冒险》（精校版全本）作者：国王陛下_utf8/summary_trimmed.txt
2025-05-03 14:30:20,531 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/30000_outline_1000/《全球进化》（精校版全本）作者：咬狗_utf8/summary_trimmed.txt
2025-05-03 14:30:33,232 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/30000_outline_1000/《武林半侠传》（校对版全本）作者：文抄公_utf8/summary_trimmed.txt
2025-05-03 14:30:43,921 - INFO - ✅ Written: /content/json_results/jso


🎉 All done! Outputs saved to /content/json_results/json_gemini_normalized/30000_outline_1000


In [55]:
!rm -rf /content/json_results/json_gemini_normalized/30000_outline_1000

In [54]:
summarize_outlines_batch(
    input_dir="/content/json_results/json_gemini_normalized/100000_summary_gathered",
    input_filename="entire_section_outline.txt",
    output_dir="/content/json_results/json_gemini_normalized/100000_outline_10000",
    output_filename="summary_trimmed.txt",
    prompt_text=prompt_10000,
    gemini_api_key="your-default-api-key"
)

summarize_outlines_batch(
    input_dir="/content/json_results/json_gemini_normalized/100000_summary_gathered",
    input_filename="entire_section_outline.txt",
    output_dir="/content/json_results/json_gemini_normalized/100000_outline_5000",
    output_filename="summary_trimmed.txt",
    prompt_text=prompt_5000,
    gemini_api_key="your-default-api-key"
)

summarize_outlines_batch(
    input_dir="/content/json_results/json_gemini_normalized/100000_summary_gathered",
    input_filename="entire_section_outline.txt",
    output_dir="/content/json_results/json_gemini_normalized/100000_outline_1000",
    output_filename="summary_trimmed.txt",
    prompt_text=prompt_1000,
    gemini_api_key="your-default-api-key"
)

📚 Summarizing Outlines:   0%|          | 0/40 [00:00<?, ?it/s]

2025-05-03 11:02:54,609 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/100000_outline_10000/《奋斗在新明朝》（校对版全本）作者：随轻风去_utf8/summary_trimmed.txt
2025-05-03 11:03:28,911 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/100000_outline_10000/《反正我是超能力者》（校对版全本）作者：吃书妖_utf8/summary_trimmed.txt
2025-05-03 11:03:43,714 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/100000_outline_10000/《天可汗》（校对版全本）作者：西风紧_utf8/summary_trimmed.txt
2025-05-03 11:04:17,004 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/100000_outline_10000/《崩坏世界的传奇大冒险》（精校版全本）作者：国王陛下_utf8/summary_trimmed.txt
2025-05-03 11:04:36,075 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/100000_outline_10000/《全球进化》（精校版全本）作者：咬狗_utf8/summary_trimmed.txt
2025-05-03 11:04:51,199 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/100000_outline_10000/《武林半侠传》（校对版全本）作者：文抄公_utf8/summary_trimmed.txt
2025-05-03 11:05:11,754 - INFO - ✅ Written: /content/json


🎉 All done! Outputs saved to /content/json_results/json_gemini_normalized/100000_outline_10000


📚 Summarizing Outlines:   0%|          | 0/40 [00:00<?, ?it/s]

2025-05-03 11:18:32,586 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/100000_outline_5000/《奋斗在新明朝》（校对版全本）作者：随轻风去_utf8/summary_trimmed.txt
2025-05-03 11:18:58,143 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/100000_outline_5000/《反正我是超能力者》（校对版全本）作者：吃书妖_utf8/summary_trimmed.txt
2025-05-03 11:19:37,091 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/100000_outline_5000/《天可汗》（校对版全本）作者：西风紧_utf8/summary_trimmed.txt
2025-05-03 11:19:53,281 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/100000_outline_5000/《崩坏世界的传奇大冒险》（精校版全本）作者：国王陛下_utf8/summary_trimmed.txt
2025-05-03 11:20:05,627 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/100000_outline_5000/《全球进化》（精校版全本）作者：咬狗_utf8/summary_trimmed.txt
2025-05-03 11:20:21,087 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/100000_outline_5000/《武林半侠传》（校对版全本）作者：文抄公_utf8/summary_trimmed.txt
2025-05-03 11:20:38,917 - INFO - ✅ Written: /content/json_resul


🎉 All done! Outputs saved to /content/json_results/json_gemini_normalized/100000_outline_5000


📚 Summarizing Outlines:   0%|          | 0/40 [00:00<?, ?it/s]

2025-05-03 11:32:19,748 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/100000_outline_1000/《奋斗在新明朝》（校对版全本）作者：随轻风去_utf8/summary_trimmed.txt
2025-05-03 11:32:31,299 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/100000_outline_1000/《反正我是超能力者》（校对版全本）作者：吃书妖_utf8/summary_trimmed.txt
2025-05-03 11:33:01,650 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/100000_outline_1000/《天可汗》（校对版全本）作者：西风紧_utf8/summary_trimmed.txt
2025-05-03 11:33:14,863 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/100000_outline_1000/《崩坏世界的传奇大冒险》（精校版全本）作者：国王陛下_utf8/summary_trimmed.txt
2025-05-03 11:33:28,179 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/100000_outline_1000/《全球进化》（精校版全本）作者：咬狗_utf8/summary_trimmed.txt
2025-05-03 11:33:53,035 - INFO - ✅ Written: /content/json_results/json_gemini_normalized/100000_outline_1000/《武林半侠传》（校对版全本）作者：文抄公_utf8/summary_trimmed.txt
2025-05-03 11:34:08,834 - INFO - ✅ Written: /content/json_resul


🎉 All done! Outputs saved to /content/json_results/json_gemini_normalized/100000_outline_1000


In [51]:
!rm -rf /content/json_results/json_gemini_normalized/100000_outline_10000
!rm -rf /content/json_results/json_gemini_normalized/100000_outline_1000
!rm -rf /content/json_results/json_gemini_normalized/100000_outline_5000